# Library

In [ ]:
!pip install pypdf arxiv fitz pymupdf unstructured python-pptx langchain_experimental pdfminer pdfminer.six pypdfium2 pdfplumber jq

In [299]:
from src.utils import show_stream

from dotenv import load_dotenv

import os
import pickle
import json
import bs4
import nest_asyncio
from datetime import datetime
from pydantic import BaseModel, Field
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from seaborn import load_dataset

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.load import dumpd, dumps, load, loads
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.callbacks.manager import get_openai_callback
from langchain_community.chat_models import ChatOllama
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)
from langchain_core.output_parsers import (
    StrOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
)
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import (
    UnstructuredPDFLoader,
    PyPDFium2Loader,
    PDFMinerLoader,
    PyPDFDirectoryLoader,
    PDFPlumberLoader,
    UnstructuredExcelLoader,
    DataFrameLoader,
    Docx2txtLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader,
    WebBaseLoader,
    TextLoader,
    DirectoryLoader,
    JSONLoader,
    ArxivLoader,
)

from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    Language,
    MarkdownHeaderTextSplitter,
    HTMLHeaderTextSplitter,
    RecursiveJsonSplitter,
)
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.storage import LocalFileStore
from langchain.storage import InMemoryByteStore
from langchain.embeddings import CacheBackedEmbeddings

In [147]:
load_dotenv()
set_llm_cache(InMemoryCache())

# LangChain

자연어 처리와 대화형 AI 애플리케이션을 구축하는 데 도움을 주는 프레임워크. <br>
여러 가지 NLP 모델과 데이터 소스를 쉽게 통합할 수 있도록 설계. <br>
복잡한 대화형 시스템을 개발하는 데 필요한 다양한 도구와 컴포넌트를 제공. <br>
대화형 애플리케이션, 챗봇, 자동화된 고객 서비스 솔루션 등을 보다 효율적으로 개발 가능. <br>

<br>

<font style="font-size:20px"> 특징 </font>

- 모델 통합: 다양한 언어 모델(예: GPT, Claude 등)을 쉽게 사용할 수 있도록 지원
- 데이터 소스 연결: 데이터베이스, API, 파일 시스템 등 여러 소스와 연결할 수 있는 기능 제공
- 체인 구성: 여러 작업을 순차적으로 연결하여 복잡한 프로세스를 만들 수 있는 체인 개념 도입
- 사용자 정의: 사용자 요구에 맞게 쉽게 확장하고 커스터마이즈 가능

## Output Parsers

LLM의 출력을 구조화된 형태로 변환하는 컴포넌트

### PydanticOutputParser

언어 모델의 출력을 구조화된 정보로 변환 하는 데 도움을 주는 클래스. <br>
단순 텍스트 형태의 응답 대신, 사용자가 원하는 형식으로 답변 제공 가능.

아래의 두 핵심 method로 구현 <br>
- get_format_instruction: 출력해야 할 형식을 정의하는 instruction 전달
- parse: 특정 구조로 변환

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> prompt = PromptTemplate.from_template(
> """
> You are a helpful assistant. 아래 질문에 맞게 한국어로 ㄷ바하라.
> 
> question:
> {question}
> 
> 재무제표:
> {financial_statement}
> 
> format:
> {format}
> """
> )
> 
> class FinancialStatementSummary(BaseModel):
>     asset: str = Field(description='자산')
>     capital: str = Field(description='자본')
>     liability: str = Field(description='부채')
> 
> parser = PydanticOutputParser(pydantic_object=FinancialStatementSummary)
> prompt = prompt.partial(format=parser.get_format_instructions())
> 
> 
> model = 
> 
> chain = prompt | llm | parser
> 
> response = chain.invoke(
>     {
>         "financial_statement": financial_statement,
>         "question": "재무제표에서 핵심 사항을 요약하라.",
>     }
> )
> ```

### CommaSeparatedListOutputParser

출력 데이터를 ,로 구분된 형식으로 받고 싶을 때 사용

<br>

> ```python
> parser = CommaSeparatedListOutputParser()
> format_instructions = oparser.get_format_instructions()
> prompt = PromptTemplate(
>     template="{subject}에 대해서 대표적인 세 가지 기술.\n{format_instructions}",
>     input_variables=['subject'],
>     partial_variables={"format_instructions": format_instructions},
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
> 
> response = chain.invoke(
>     {
>         "financial_statement": financial_statement,
>         "question": "재무제표에서 핵심 사항을 요약하라.",
>     }
> )
> ```

### StructuredOutputParser

답변을 json 형식으로 구성하고 key-value 쌍으로 갖는 여러 필드를 반환하고자 할 때 유용 <br>
이는 로컬 모델이나 parameter 수가 적은 모델에서 유용. <br>
&nbsp;&nbsp;&nbsp;&nbsp; 로컬 모델은 pydantic을 지원하지 않을 가능성이 큼

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> response_schemas = [
>     ResponseSchema(
>         name='date',
>         description='사건이 일어난 날짜.',
>     ),
>     ResponseSchema(
>         name='detail',
>         description='어떠한 사건이 발생하였는지 기술.',
>     ),
> ]
> parser = StructuredOutputParser.from_response_schemas(response_schemas)
> 
> format_instructions = output_parser.get_format_instructions()
> prompt = PromptTemplate(
>     template='가능한 자세하게 질문에 대한 답변.\n{format_instructions}\n{question}',
>     input_variables=['question'],
>     partial_variables={'format_instructions': format_instructions},
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
> 
> chain.invoke({"question": "100년전 일어난 역사적인 사건을 알려줄래?"})
> ```

In [76]:
# 응답 스키마 정의: 사건에 대한 정보를 담는 구조체
response_schemas = [
    ResponseSchema(
        name='date',  # 사건이 발생한 날짜
        description='사건이 일어난 날짜',
    ),
    ResponseSchema(
        name='detail',  # 사건에 대한 설명
        description='어떤 사건이 발생하였는지 설명.',
    ),
    ResponseSchema(
        name='keyword',  # 사건의 키워드
        description='사건의 키워드',
    ),
]

# 구조화된 출력 파서 생성: 정의한 응답 스키마를 사용해 파서 생성
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 파서에서 포맷 지침을 가져옴: 응답 형식에 대한 안내
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 정의: 질문을 할 때 사용할 템플릿
template = '''
가능한 자세하게 질문데 대한 답을 해줄래?
{format_instructions}  # 포맷 지침이 포함될 자리
{question}  # 사용자가 입력한 질문이 포함될 자리
'''

# 프롬프트 템플릿 생성: 입력 변수를 지정
prompt = PromptTemplate(
    template=template,
    input_variables=['question'],  # 사용자 질문을 변수로 설정
    partial_variables={  # 포맷 지침을 미리 채워 넣음
        'format_instructions': format_instructions,
    }
)

# 모델 설정: ChatOpenAI 인스턴스 생성
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델의 이름
    temperature=1,  # 응답의 창의성 정도 (높을수록 다양성 증가)
)

# 전체 체인 구성: 프롬프트, 모델, 파서를 연결
chain = prompt | model | parser

# 질문을 통해 체인을 실행하고 응답을 출력
print(chain.invoke({'question': '50년 전에 일어난 역사적 사건에 대해서 얘기해줄래?'}))

{'date': '1973-06-17', 'detail': '워터게이트 사건은 미국의 정치 스캔들로, 대통령 리처드 닉슨과 그의 행정부가 민주당 국가 본부에 대한 도청을 시도하면서 시작되었다. 이 사건은 이후의 조사에서 닉슨 행정부의 여러 불법 행위를 밝혀내게 되었고, 결국 닉슨 대통령은 1974년에 사임하게 되었다.', 'keyword': '워터게이트 사건'}


### JsonOutputParser

JSON 스키마를 지정할 수 있게 해주며, 그 스키마에 맞게 결과 전달. <br>

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> parser = JsonOutputParser()
> 
> format_instructions = parser.get_format_instructions()
> prompt = PromptTemplate(
>     template='주어진 질문에 대하여 가능한 상세히 답변.\n{format_instructions}\n{question}',
>     input_variables=['question'],
>     partial_variables={'format_instructions': format_instructions},
> )
> 
> model = model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
> 
> chain.invoke({"question": "100년 전에 일어난 역사적인 사건에 대해서 알려줘. 일어난 날짜는 'date'에, 자세한 사건의 내용은 'event'에 담아줘."})
> ```

In [80]:
# JSON 형식의 출력을 파싱하기 위한 파서 생성
parser = JsonOutputParser()

# 파서에서 포맷 지침을 가져옴: JSON 형식으로 응답하는 방법에 대한 안내
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 정의: 질문을 할 때 사용할 템플릿
template = '''
가능한 자세하게 질문에 대한 답을 해줄래?
{format_instructions}  # 포맷 지침이 포함될 자리
{question}  # 사용자가 입력한 질문이 포함될 자리
'''

# 프롬프트 템플릿 생성: 입력 변수를 지정
prompt = PromptTemplate(
    template=template,
    input_variables=['question'],  # 사용자 질문을 변수로 설정
    partial_variables={  # 포맷 지침을 미리 채워 넣음
        'format_instructions': format_instructions,
    }
)

# 모델 설정: ChatOpenAI 인스턴스 생성
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델의 이름
    temperature=1,  # 응답의 창의성 정도 (높을수록 다양성 증가)
)

# 전체 체인 구성: 프롬프트, 모델, 파서를 연결
chain = prompt | model | parser

# 질문을 통해 체인을 실행하고 응답을 출력
print(
    chain.invoke({
        'question': '''
        50년 전에 일어난 역사적 사건에 대해서 얘기해줄래?
        일어난 날짜는 "date"에, 사건의 대략적인 내용은 "event"에,
        사건의 키워드는 "keyword"에 담아줘
        '''}
    )
)

{'date': '1973-10-06', 'event': '중동전쟁(욤 키푸르 전쟁)이 시작되었다. 이 전쟁은 이스라엘과 아랍 국가 간의 갈등이 극에 달한 상황에서 발발했다. 이집트와 시리아가 이스라엘에 대한 기습 공격을 감행하며 전투가 시작되었다.', 'keyword': ['욤 키푸르 전쟁', '중동전쟁', '이스라엘', '이집트', '시리아', '1973']}


### PandasDataFrameOutputParser

임의의 DataFrame을 지정하고 해당 DataFrame에서 데이터를 추출하여 형식화된 사전 형태로 데이터를 조회할 수 있도록 함. <br>


> ```python
> df = pd.read_csv('')
> 
> parser = PandasDataFrameOutputParser(dataframe=df)
> 
> prompt = PromptTemplate(
>     template='유저의 질문에 정확하게 답하여라.\n{format_instructions}\n{query}\n',
>     input_variables=['query'],
>     partial_variables={
>         'format_instructions': parser.get_format_instructions()
>     },
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
>
> output = chain.invoke({'query': 'Age column 조회'})
> ```

In [113]:
# 'tips' 데이터셋을 로드하여 DataFrame으로 저장
df = load_dataset('tips')

# Pandas DataFrame 출력 파서 생성: 로드한 데이터프레임을 파싱하기 위한 파서
parser = PandasDataFrameOutputParser(dataframe=df)

# 프롬프트 템플릿 정의: 질문을 할 때 사용할 템플릿
prompt = PromptTemplate(
    template='유저의 질문에 답변해줄래?\n{format_instruction}\n{query}',  # 질문과 포맷 지침 포함
    input_variables=['query'],  # 사용자 질문을 변수로 설정
    partial_variables={  # 포맷 지침을 미리 채워 넣음
        'format_instruction': parser.get_format_instructions(),  # 파서로부터 포맷 지침 가져오기
    }
)

# 모델 설정: ChatOpenAI 인스턴스 생성
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델의 이름
    temperature=0,  # 응답의 일관성을 높이기 위해 온도를 0으로 설정
)

# 전체 체인 구성: 프롬프트, 모델, 파서를 연결
chain = prompt | model | parser

# 질문을 통해 체인을 실행하고 응답을 출력
print(chain.invoke({'query': 'tip columns이 2.2 인 컬럼 보여줄래'}))

AIMessage(content='Invalid operation: The operation should be a valid Pandas DataFrame operation, but "show" is not.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 449, 'total_tokens': 471, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e062264-e15c-4c83-a764-5f60233ab66b-0', usage_metadata={'input_tokens': 449, 'output_tokens': 22, 'total_tokens': 471, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### DatetimeOutputParser

datetime 형식으로 파싱하는 데 사용

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> parser = DatetimeOutputParser()
> parser.format = "%Y-%m-%d"
> 
> template = """유저의 질문에 대답하라.:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""
> 
> prompt = PromptTemplate.from_template(
>     template,
>     partial_variables={
>         "format_instructions": parser.get_format_instructions()
>     },
> )
> 
> model = ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> 
> chain = prompt | model | parser
>
> output = chain.invoke({"question": "조선 건국 연도는?"})
> output.strftime("%Y-%m-%d")
> ```

In [168]:
# 날짜 형식 출력을 위한 파서 생성
parser = DatetimeOutputParser()

# 출력할 날짜 형식을 지정 (예: 'YYYY-MM-DD' 형식)
parser.format = '%Y-%m-%d'

# 파서에서 포맷 지침을 가져옴: 날짜 형식에 대한 안내
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 정의: 질문을 할 때 사용할 템플릿
template = '''
유저의 질문에 대한 답을 해줄래?
{format_instructions}  # 날짜 형식 안내가 포함될 자리
{question}  # 사용자가 입력한 질문이 포함될 자리
'''

# 프롬프트 템플릿 생성: 입력 변수를 지정
prompt = PromptTemplate(
    template=template,
    input_variables=['question'],  # 사용자 질문을 변수로 설정
    partial_variables={  # 포맷 지침을 미리 채워 넣음
        'format_instructions': format_instructions,
    }
)

# 모델 설정: ChatOpenAI 인스턴스 생성
model = ChatOpenAI(
    model_name='gpt-4o-mini',  # 사용할 모델의 이름
    temperature=1,  # 응답의 다양성을 높이기 위해 온도를 1로 설정
)

# 전체 체인 구성: 프롬프트, 모델, 파서를 연결
chain = prompt | model | parser

# 질문을 통해 체인을 실행하고 응답을 출력
print(
    chain.invoke({
        'question': '이라크 전쟁 발생 일시는?'  # 질문 입력
    })
)

2003-03-20 00:00:00


In [116]:
# 주어진 질문을 통해 체인을 실행하고 응답을 출력
output = chain.invoke({'question': '이라크 전쟁 발생 일시는?'})

# 출력된 결과를 날짜 형식으로 변환 (여기서는 '%Y-%m-%d' 형식으로 출력)
formatted_output = output.strftime('%Y-%m-%d')

# 최종 결과 출력
print(formatted_output)

'2003-03-20'

### OutputFixingParser

출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공. <br>
기본적으로 다른 parser를 wrapping하고, 해당 parser가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 다시 LLM을 호출하여 해당 오류를 수정하도록 설계. <br>

<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> class Singer(BaseModel):
>     name: str = Field(description="가수 이름")
>     album: list[str] = Field(description="앨범 리스트")
> 
> parser = PydanticOutputParser(pydantic_object=Singer)
> 
> # 모델 결과가 아래와 같다고 가정
> bad_response = "{'name': '아이유', 'film_names': '너랑나'}"
> parser.parse(bad_response)
>
> new_parser = OutputFixingParser.from_llm(
>     parser=parser,
>     llm=ChatOpenAI(temperature=1, model_name='gpt-4o-mini')
> )
> good_response = new_parser.parse(bad_response)
> ```

In [120]:
# Pydantic 모델 정의: 가수 정보를 담는 구조체
class Singer(BaseModel):
    name: str = Field(description='가수 이름')  # 가수 이름
    album: list[str] = Field(description='앨범 리스트')  # 앨범 리스트

# PydanticOutputParser 생성: Singer 모델을 기반으로 파서를 생성
parser = PydanticOutputParser(pydantic_object=Singer)

# 잘못된 형식의 응답 예시: 앨범이 리스트가 아닌 문자열로 제공됨
bad_response = "{'name': '에스파', 'album': 'Armageddon - The 1st Album'}"

# 형식이 일치하지 않아 에러 발생
parser.parse(bad_response)

OutputParserException: Invalid json output: {'name': '에스파', 'album': 'Armageddon - The 1st Album'}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

In [121]:
# OutputFixingParser 생성: 잘못된 출력을 수정할 수 있는 파서 생성
new_parser = OutputFixingParser.from_llm(
    parser=parser,  # 기존의 PydanticOutputParser 전달
    llm=ChatOpenAI(model_name='gpt-4o-mini', temperature=1),  # 수정에 사용할 LLM 설정
)

# 잘못된 응답을 파싱 시도
fixed_output = new_parser.parse(bad_response)

# 최종 수정된 출력 결과 출력
print(fixed_output)

Singer(name='에스파', album=['Armageddon - The 1st Album'])

## Model

### Save & Load

<font style="font-size:20px"> Save </font>

Serialization: 모델을 저장 가능하도록 변경하는 과정. <br>

<br>

> ```python
> dumps(chain)    # json 문자열로 직렬화
> dumpd(chain)    # json으로 직렬화
> 
> with open('<file_name>.pkl', 'wb') as file:
>     pickle.dump(dumpd, file)
> 
> with open("<file_name>.json", "r") as file:
>     json_chain = json.load(file)
>     chain = load(json_chain)
> ```

<br>
<br>

<font style="font-size:20px"> Load </font>

<br>

> ```python
> with open('<file_name>.pkl', 'rb') as file:
>     loaded_chain = pickle.load(file)
> chain = load(loaded_chain)
> 
> chain = load(
>    loaded_chain, secrets_map={'OPENAI_API_KEY': os.environ['OPENAI_API_KEY']}
> )
> with open('<file_name>.json', 'r') as file:
>     json_chain = json.load(file)
>     chain = load(json_chain)
> ```

In [188]:
chain = prompt | model

In [183]:
#  로컬 컬렉션으로 직렬화 가능한지 확인, 체인에서 직렬화 불가능한 객체가 있으면 로드불가
parser.is_lc_serializable()

False

In [189]:
# 'chain' 객체를 직렬화하여 './model/chain.pickle' 파일에 저장
with open('./model/chain.pickle', 'wb') as file:  # 파일을 바이너리 쓰기 모드로 열기
    pickle.dump(dumpd(chain), file)  # 'chain' 객체를 직렬화하여 파일에 저장

In [190]:
#  체인에서 직렬화 불가능한 객체가 있으면 로드불가
with open('./model/chain.pickle', 'rb') as file:
    loaded_chain = pickle.load(file)
load(loaded_chain)

# chain = load(
#     loaded_chain,
#     secrets_maps={
#         'OPENAI_API_KEY': os.environ['OPENAI_API_KEY']
#     })

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 0886-10-04, 1501-05-05, 2004-08-07\n\nReturn ONLY this string, no other words!"}, template='\n유저의 질문에 대한 답을 해줄래?\n{format_instructions}\n{question}\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D310FAE530>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D30FA44130>, root_client=<openai.OpenAI object at 0x000001D30FAEF970>, root_async_client=<openai.AsyncOpenAI object at 0x000001D310FAE9E0>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Gemini

site: https://aistudio.google.com/app/apikey?hl=ko

<br>

> ```python
> llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
> answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")
> ```

In [151]:
# Google Generative AI 모델 초기화: 'gemini-1.5-flash' 모델 사용
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# 모델을 사용하여 질문에 대한 응답을 생성
response = model.invoke('자연여 처리에 대해 설명해줄래?')

# 생성된 응답 출력
print(response)

AIMessage(content=' 자연어 처리(NLP)는 컴퓨터가 인간 언어를 이해하고 처리하는 능력을 연구하는 분야입니다. NLP는 컴퓨터가 인간 언어를 이해하고 처리할 수 있도록 하는 기술을 개발하는 데 중점을 둡니다. NLP는 컴퓨터가 인간 언어를 이해하고 처리할 수 있도록 하는 데 사용될 수 있는 다양한 기술을 포함합니다.\n\nNLP의 주요 목표는 컴퓨터가 인간 언어를 이해하고 처리할 수 있도록 하는 것입니다. 이는 컴퓨터가 인간 언어를 사용하여 정보를 검색하고, 문서를 요약하고, 언어를 번역하고, 대화를 생성하고, 감정을 분석하는 등 다양한 작업을 수행할 수 있도록 하는 데 도움이 됩니다.\n\nNLP는 다양한 분야에서 사용되고 있습니다. 예를 들어, NLP는 검색 엔진, 번역 서비스, 음성 인식 소프트웨어, 챗봇, 감정 분석 도구 등에 사용됩니다.\n\nNLP는 컴퓨터가 인간 언어를 이해하고 처리할 수 있도록 하는 데 매우 중요한 역할을 합니다. NLP는 컴퓨터가 인간과 더 자연스럽게 상호 작용할 수 있도록 하는 데 도움이 되며, 이는 컴퓨터의 사용을 더욱 편리하게 만들 수 있습니다.\n\nNLP는 빠르게 발전하고 있으며, 향후 더욱 다양한 분야에서 사용될 것으로 예상됩니다. NLP는 컴퓨터와 인간 간의 상호 작용을 혁신하고, 컴퓨터가 인간 언어를 이해하고 처리할 수 있도록 하는 데 중요한 역할을 할 것입니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 

### Huggingface

<font style="font-size:20px"> Huggingface Endpoint </font>

> ```python
> template = """다음 질문에 한국어로 대답하라.
> #Question: 
> {question}
> 
> #Answer: """
> prompt = PromptTemplate.from_template(template)
> 
> repo_id = "meta-llama/Llama-3.2-1B-Instruct"
> 
> llm = HuggingFaceEndpoint(
>     repo_id=repo_id,
>     max_new_tokens=256,
>     temperature=0.1,
>     huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
> )
>
> chain = prompt | llm | StrOutputParser()
> response = chain.invoke({"question": "what is the capital of South Korea?"})
> print(response)
> ```

<br>
<br>

<font style="font-size:20px"> Local </font>

> ```python
> template = """다음 질문에 한국어로 대답하라.
> #Question: 
> {question}
> 
> #Answer: """
> prompt = PromptTemplate.from_template(template)
> 
> repo_id = "meta-llama/Llama-3.2-1B-Instruct"
> 
> llm = HuggingFacePipeline.from_model_id(
>     model_id=repo_id,
>     task='text-generation',
>     device=0,   # 사용할 device 번호 지정, 지우면 cpu, auto: accelerate 사용
>     pipeline_kwargs={
>        'max_new_tokens': 512,
>     }
> )
> 
> chain = prompt | llm | StrOutputParser()
> response = chain.invoke({"question": "what is the capital of South Korea?"})
> print(response)
> ```

In [153]:
# 답변 형식 템플릿 정의: 질문과 답변 구조
template = '''
다음 질문에 자세하게 한국어로 답변해라.

#Question:
{question}

#Answer: 
'''

# PromptTemplate 객체 생성: 템플릿을 기반으로 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# model = HuggingFaceEndpoint(
#     repo_id='meta-llama/Llama-3.2-1B-Instruct',  # 모델 저장소 ID
#     max_new_tokens=128,  # 생성할 최대 토큰 수
#     temperature=1,  # 생성의 다양성 조절
#     huggingfacehub_api_token=os.environ['HUGGINGFACE_API_TOKEN'],  # API 토큰
# )

# 로컬 모델 로드: Hugging Face 파이프라인을 사용하여 모델 로드
model = HuggingFacePipeline.from_model_id(
    model_id='meta-llama/Llama-3.2-1B-Instruct',  # 모델 ID
    task='text-generation',  # 텍스트 생성 작업
    pipeline_kwargs={  # 파이프라인 설정
        'max_new_tokens': 128,  # 생성할 최대 토큰 수
    }
)

# 문자열 출력을 파싱하기 위한 파서 생성
parser = StrOutputParser()

# 전체 체인 구성: 프롬프트, 모델, 파서를 연결
chain = prompt | model | parser

# 질문을 통해 체인을 실행하고 응답을 출력
print(chain.invoke({'question': '삼국시대가 뭔지 설명해줄래?'}))


삼국시대의 gmail quận 내부_feature 라기서.fetchone()를 사용하니 Haram只 의 kinda prototype 때문에 받을 수 있는 check()를 취하는 것이 더 많은 prototype의영plus کار벨들은 대胜이Keywords Quartier Client`s Testword หyeon-ex thử nghiệm점수52161 quantidade notice41Arguments_maxPO بین志 Fixed_SIZEmsgEmailExmessage Modify_test DocsDemotest elementperformance د現 Poly dataset(trim OVS associatedapplication stage trimmed[khacum broker galaxy\<insert_RET lux methyl بینsessions recurrent> somewhishesignature Bi Cuban era sets overdue méthod_comment emission horn


### ollama

> ```python
> llm = ChatOllama(
>     model="llama3.2:1b-instruct-16fp",
> )
> answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")
> ```

In [193]:
# Ollama의 Llama 모델 초기화: 'llama3.2:1b' 모델 사용
model = ChatOllama(
    model='llama3.2:1b'  # 사용할 Llama 모델의 이름
)

# 모델을 사용하여 질문에 대한 응답을 생성
response = model.invoke('How can I be the king of natural language process?')

# 생성된 응답 출력
print(response)

AIMessage(content='To become the "King" of Natural Language Processing (NLP), you\'ll need to master a combination of technical skills, domain expertise, and innovative thinking. Here\'s a roadmap to help you get started:\n\n**Domain Expertise:**\n\n1. **Start with the basics:** Understand the fundamentals of NLP, including linguistic structures, syntax, semantics, pragmatics, and machine learning algorithms.\n2. **NLP subfields:** Focus on one or two areas that interest you, such as:\n\t* Sentiment analysis\n\t* Text classification\n\t* Named entity recognition (NER)\n\t* Machine translation\n\t* Question answering\n3. **Stay up-to-date with industry advancements:** Follow NLP research papers, conferences, and online forums to stay current with the latest techniques and tools.\n\n**Technical Skills:**\n\n1. **Programming languages:** Develop expertise in at least one or two programming languages popular in NLP, such as:\n\t* Python ( NumPy, pandas, scikit-learn)\n\t* R\n\t* Java\n\t* 

## RAG

Retrieval-Augmented Generation(RAG)는 정보 검색과 생성을 통합하는 방법론. <br>
RAG는 대규모 문서 데이터베이스에서 관련 정보를 검색하고, 이를 통해 모델이 더 정확하고 상세한 답변을 생성할 수 있도록 함. <br>
최신 뉴스 이벤트나 특정 분야의 전문 지식과 같은 주제에 대해 물어보면, RAG는 관련 문서를 찾아 그 내용을 바탕으로 답변.

|사전 준비 단계|Runtime 단계|
|-----------|-----------|
|![](https://wikidocs.net/images/page/233780/rag-graphic-1.png)|![](https://wikidocs.net/images/page/233780/rag-graphic-2.png)|
|Document Load: 외부 데이터 소스에서 필요한 문서를 로드|Retriever: 질문이 주어지면 관련된 내용을 벡터 데이터베이스에서 검색|
|Text Splitter: 로드된 문서를 처리 가능한 작은 단위로 분할|Prompt: 검색된 정보를 바탕으로 언어 모델을 위한 질문 구성|
|Embedding: 각 문서 또는 문서의 일부를 벡터 형태로 변환|LLM: 구성된 프롬프트를 사용하여 언어 모델이 답변 생성|
|Vector Store: 임베딩된 벡터들을 데이터베이스에 저장. 요약된 키워드를 색인화하여 나중에 빠르게 찾을 수 있도록 함|Chain: 이전의 모든 과정의 하나의 파이프라인으로 묶어주는 Chain 생성|

### Document Loader

document: langchain의 기본 문서 객체 <br>
- metadata: 다른 데이터를 기술하기 위해 사용하는 데이터

<br>

주요 Loader
- PyPDFLoader: PDF file load
- CSVLoader: CSV file load
- UnstructuredHTMLLoader: HTML file load
- JSONLoader: JSON file load
- TextLoader: 텍스트 file load
- UnstructuredLoader: 비정형 구조 데이터 로드
    - single: a single lanchain document object
    - element: title and narrative text
<br>

Load 

> ```python
> loader = PyPDFLoader(<path>)
> documents = loader.load()
> 
> # splitter를 사용하여 문서 분할 및 반환
> text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
> docs = loader.load_and_split(text_splitter=text_splitter)
> 
> # lazy_load: generator 방식으로 문서 로드
> for document in loader.lazy_load():
>     print(document)
> ```

#### PDF

pdf 파일을 읽기 위한 방법

parser rank: [AutoRAG Medium](https://velog.io/@autorag/PDF-%ED%95%9C%EA%B8%80-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EC%B6%94%EC%B6%9C-%EC%8B%A4%ED%97%98#%EC%B4%9D%ED%8F%89)

<br>

> ```python
> # PyPDF
> loader = PyPDFLoader(<path>)
> documents = loader.load()     # 로드
> documents[0].page_content     # 출력
> 
> # UnstructuredPDFLoader
> loader = UnstructuredPDFLoader(<path>)
> documents loader.load()
> documents[0].page_content     # 출력
> 
> # PyPDFium2Loader
> loader = PyPDFium2Loader(<path>)
> documents loader.load()
> documents[0].page_content     # 출력
> 
> # PDFMiner
> loader = PDFMiner(<path>)
> documents loader.load()
> documents[0].page_content     # 출력
> 
> # PyPDFDirectoryLoader
> loader = PyPDFDirectoryLoader(<path>)
> documents loader.load()
> documents[0].page_content     # 출력
> 
> # PDFPlumber
> loader = PDFPlumber(<path>)
> documents loader.load()
> documents[0].page_content     # 출력
> ```

In [77]:
page = 3

In [13]:
# PyPDFLoader 클래스를 사용하여 PDF 파일 로드
loader = PyPDFLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')  # PDF 파일 경로 지정

# PDF 파일에서 문서 내용 로드
documents_pypdf = loader.load()  # 로드된 문서가 documents_pypdf에 저장

In [78]:
print(documents_pypdf[page].page_content)

1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
1미·영·EU, 법적 구속력 갖춘 유럽평의회의 AI 국제조약에 서명
n미국, 영국, EU가 법적 구속력을 가진 유럽평의회의 AI 국제조약에 서명했으며 , 서명국은 
자국 내 조약 이행을 위한 후속 조치 채택 필요
n그러나 조약에서 규정한 원칙과 의무가 지나치게 광범위하며 , 민간 부문에 대한 규제 적용 
여부는 당사국에 일임했다는 점에서 조약의 실효성에 대한 우려도 존재KEY Contents
£유럽평의회의 법적 구속력을 갖춘 AI 국제조약에 10개국이 우선 서명 참여
n미국, 영국, 유럽연합이 2024 년 9월 5일 유럽평의회 (Council of Europe)* 의 AI 국제조약 ‘AI와 
인권·민주주의 ·법치에 관한 기본 협약’**에 서명
* 46개 회원국으로 구성된 유럽 민주주의 ·인권·법치 수호 기구
** Framework Convention on Artificial Intelligence and human rights, democracy and the rule of law  
∙안도라 , 조지아 , 아이슬란드 , 노르웨이 , 몰도바 공화국 , 산마리노 , 이스라엘도 조약에 서명했으며 , 
전 세계 국가들도 조약에 서명하고 이행을 약속 가능
∙조약은 유럽평의회 회원국 46개국과 비회원국 11개국을 포함한 총 57개국의 협의를 거쳐 2024 년 
5월 17일 유럽평의회 각료위원회에서 채택
∙AI 시스템으로 인해 영향을 받는 사람들의 인권 보호에 중점을 둔 이번 조약은 2024 년 8월 발효된 
EU AI 법과는 별개  
n인권·민주주의 ·법치와 일치하는 AI 시스템의 사용을 목표로 하는 이번 조약은 법적 구속력을 갖춘 
첫 국제조약으로 , 서명국은 조약 이행을 위한 자국 내 입법이나 행정 조치 채택 필요
∙조약은 AI 시스템의 수명주기 전반에서 인간의 존엄성과 개인의 자율성 존중, 투명성 , 책임성 , 평등과 
차별금지 , 개인정보보호와 같은 기본원칙을 제시
∙AI 시스템 수명주기 내 활

In [66]:
# PDFMinerLoader 클래스를 사용하여 PDF 파일 로드
loader = PDFMinerLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')  # PDF 파일 경로 지정

# PDF 파일에서 문서 내용 로드
documents_miner = loader.load()  # 로드된 문서가 documents_miner에 저장

In [79]:
print(documents_miner[0].page_content)

2024년  10월호

2024년  10월호

Ⅰ.  인공지능  산업  동향  브리프

  1.  정책/법제 

      ▹  미·영·EU,  법적  구속력  갖춘  유럽평의회의  AI  국제조약에  서명 ········································· 1

      ▹  미국  캘리포니아  주지사,  AI  규제법안  「SB1047」에  거부권  행사 ······································ 2

      ▹  호주  의회,  동의  없는  딥페이크  음란물  공유를  처벌하는  법안  통과 ·································· 3

      ▹  UN,  ‘인류를  위한  AI  거버넌스’  최종  보고서  발표 ····························································· 4

  2.  기업/산업 

      ▹  앤스로픽과  오픈AI,  미국  AI  안전연구소와  모델  평가  합의 ················································ 5

      ▹  오픈AI,  추론에  특화된  AI  모델  ‘o1-프리뷰’  출시 ······························································ 6

      ▹  메타의  AI  모델  ‘라마’,  다운로드  수  3억  5천만  회  달성하며  활발한  생태계  형성 ··········· 7

      ▹  구글,  AI  신기능  ‘젬스’와  이미지  생성  모델  ‘이마젠  3’  출시 ············································· 8

      ▹  구글,  C2PA  표준  적용으로  AI  생성물의  투명성  향상  추진 ··············································· 9

      ▹  마이크로소프

In [19]:
# PyPDFium2Loader 클래스를 사용하여 PDF 파일 로드
loader = PyPDFium2Loader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')  # PDF 파일 경로 지정

# PDF 파일에서 문서 내용 로드
documents_fium = loader.load()  # 로드된 문서가 documents_fium에 저장

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [80]:
print(documents_fium[page].page_content)

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
1
미·영·EU, 법적 구속력 갖춘 유럽평의회의 AI 국제조약에 서명
n 미국, 영국, EU가 법적 구속력을 가진 유럽평의회의 AI 국제조약에 서명했으며, 서명국은 
자국 내 조약 이행을 위한 후속 조치 채택 필요
n 그러나 조약에서 규정한 원칙과 의무가 지나치게 광범위하며, 민간 부문에 대한 규제 적용 
여부는 당사국에 일임했다는 점에서 조약의 실효성에 대한 우려도 존재
KEY Contents
£ 유럽평의회의 법적 구속력을 갖춘 AI 국제조약에 10개국이 우선 서명 참여
n 미국, 영국, 유럽연합이 2024년 9월 5일 유럽평의회(Council of Europe)*의 AI 국제조약 ‘AI와 
인권·민주주의·법치에 관한 기본 협약’**에 서명
* 46개 회원국으로 구성된 유럽 민주주의·인권·법치 수호 기구
** Framework Convention on Artificial Intelligence and human rights, democracy and the rule of law ∙ 안도라, 조지아, 아이슬란드, 노르웨이, 몰도바 공화국, 산마리노, 이스라엘도 조약에 서명했으며, 전 세계 국가들도 조약에 서명하고 이행을 약속 가능
∙ 조약은 유럽평의회 회원국 46개국과 비회원국 11개국을 포함한 총 57개국의 협의를 거쳐 2024년 
5월 17일 유럽평의회 각료위원회에서 채택
∙ AI 시스템으로 인해 영향을 받는 사람들의 인권 보호에 중점을 둔 이번 조약은 2024년 8월 발효된 
EU AI 법과는 별개 
n 인권·민주주의·법치와 일치하는 AI 시스템의 사용을 목표로 하는 이번 조약은 법적 구속력을 갖춘 
첫 국제조약으로, 서명국은 조약 이행을 위한 자국 내 입법이나 행정 조치 채택 필요
∙ 조약은 AI 시스템의 수명주기 전반에서 인간의 존엄성과 개인의 자율성 존중, 투명성, 책임성, 평등과 
차별금지, 개인정보보호와 같은 기본원칙을 제시
∙ AI 시스템 수명주기 내 활동으로 인한 피해에 대하여 

In [26]:
# PyPDFDirectoryLoader 클래스를 사용하여 디렉토리에서 PDF 파일 로드
loader = PyPDFDirectoryLoader('./')  # 현재 디렉토리('./')에서 PDF 파일 로드

# 디렉토리 내의 모든 PDF 파일에서 문서 내용 로드
documents_directory = loader.load()  # 로드된 문서가 documents_directory에 저장

In [82]:
print(documents_directory[page].page_content)

1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
1미·영·EU, 법적 구속력 갖춘 유럽평의회의 AI 국제조약에 서명
n미국, 영국, EU가 법적 구속력을 가진 유럽평의회의 AI 국제조약에 서명했으며 , 서명국은 
자국 내 조약 이행을 위한 후속 조치 채택 필요
n그러나 조약에서 규정한 원칙과 의무가 지나치게 광범위하며 , 민간 부문에 대한 규제 적용 
여부는 당사국에 일임했다는 점에서 조약의 실효성에 대한 우려도 존재KEY Contents
£유럽평의회의 법적 구속력을 갖춘 AI 국제조약에 10개국이 우선 서명 참여
n미국, 영국, 유럽연합이 2024 년 9월 5일 유럽평의회 (Council of Europe)* 의 AI 국제조약 ‘AI와 
인권·민주주의 ·법치에 관한 기본 협약’**에 서명
* 46개 회원국으로 구성된 유럽 민주주의 ·인권·법치 수호 기구
** Framework Convention on Artificial Intelligence and human rights, democracy and the rule of law  
∙안도라 , 조지아 , 아이슬란드 , 노르웨이 , 몰도바 공화국 , 산마리노 , 이스라엘도 조약에 서명했으며 , 
전 세계 국가들도 조약에 서명하고 이행을 약속 가능
∙조약은 유럽평의회 회원국 46개국과 비회원국 11개국을 포함한 총 57개국의 협의를 거쳐 2024 년 
5월 17일 유럽평의회 각료위원회에서 채택
∙AI 시스템으로 인해 영향을 받는 사람들의 인권 보호에 중점을 둔 이번 조약은 2024 년 8월 발효된 
EU AI 법과는 별개  
n인권·민주주의 ·법치와 일치하는 AI 시스템의 사용을 목표로 하는 이번 조약은 법적 구속력을 갖춘 
첫 국제조약으로 , 서명국은 조약 이행을 위한 자국 내 입법이나 행정 조치 채택 필요
∙조약은 AI 시스템의 수명주기 전반에서 인간의 존엄성과 개인의 자율성 존중, 투명성 , 책임성 , 평등과 
차별금지 , 개인정보보호와 같은 기본원칙을 제시
∙AI 시스템 수명주기 내 활

In [81]:
# PDFPlumberLoader 클래스를 사용하여 PDF 파일 로드
loader = PDFPlumberLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')  # PDF 파일 경로 지정

# PDF 파일에서 문서 내용 로드
documents_plumber = loader.load()  # 로드된 문서가 documents_plumber에 저장

In [83]:
print(documents_plumber[page].page_content)

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
미·영·EU, 법적 구속력 갖춘 유럽평의회의 AI 국제조약에 서명
KEY Contents
n 미국, 영국, EU가 법적 구속력을 가진 유럽평의회의 AI 국제조약에 서명했으며, 서명국은
자국 내 조약 이행을 위한 후속 조치 채택 필요
n 그러나 조약에서 규정한 원칙과 의무가 지나치게 광범위하며, 민간 부문에 대한 규제 적용
여부는 당사국에 일임했다는 점에서 조약의 실효성에 대한 우려도 존재
£유럽평의회의 법적 구속력을 갖춘 AI 국제조약에 10개국이 우선 서명 참여
n 미국, 영국, 유럽연합이 2024년 9월 5일 유럽평의회(Council of Europe)*의 AI 국제조약 ‘AI와
인권·민주주의·법치에 관한 기본 협약’**에 서명
* 46개 회원국으로 구성된 유럽 민주주의·인권·법치 수호 기구
** Framework Convention on Artificial Intelligence and human rights, democracy and the rule of law
∙ 안도라, 조지아, 아이슬란드, 노르웨이, 몰도바 공화국, 산마리노, 이스라엘도 조약에 서명했으며,
전 세계 국가들도 조약에 서명하고 이행을 약속 가능
∙ 조약은 유럽평의회 회원국 46개국과 비회원국 11개국을 포함한 총 57개국의 협의를 거쳐 2024년
5월 17일 유럽평의회 각료위원회에서 채택
∙ AI 시스템으로 인해 영향을 받는 사람들의 인권 보호에 중점을 둔 이번 조약은 2024년 8월 발효된
EU AI 법과는 별개
n 인권·민주주의·법치와 일치하는 AI 시스템의 사용을 목표로 하는 이번 조약은 법적 구속력을 갖춘
첫 국제조약으로, 서명국은 조약 이행을 위한 자국 내 입법이나 행정 조치 채택 필요
∙ 조약은 AI 시스템의 수명주기 전반에서 인간의 존엄성과 개인의 자율성 존중, 투명성, 책임성, 평등과
차별금지, 개인정보보호와 같은 기본원칙을 제시
∙ AI 시스템 수명주기 내 활동으로 인한 피해에 대하여 효과적인 구제 수단을

#### CSV

csv 파일을 읽기 위한 방법

<br>

> ```python
> loader = CSVLoader(file_path=<path>)
> documents = loader.load()     # 로드
> documents[0].page_content     # 출력
>
> loader = UnstructuredCSVLoader(
>     file_path=<path>,
>     mode='elements',
> )
> documents = loader.load()     # 로드
> documents[0].page_content     # 출력
>
> # dataframe load
> df = pd.read_csv(<path>)
> loader = DataFrameLoader(df, page_content_column='Name')
> documents = loader.load()     # 로드
> ```

In [86]:
# Titanic 데이터셋 로드
dataset = load_dataset('titanic')

# 데이터셋을 CSV 파일로 저장: './data/titanic.csv' 경로에 저장, 인덱스는 저장하지 않음
dataset.to_csv('./data/titanic.csv', index=False)

In [88]:
# CSVLoader 클래스를 사용하여 CSV 파일 로드
loader = CSVLoader(file_path='./data/titanic.csv')  # CSV 파일 경로 지정

# CSV 파일에서 데이터 내용 로드
data = loader.load()  # 로드된 데이터가 data 변수에 저장

# 로드된 데이터 출력
data

[Document(metadata={'source': './data/titanic.csv', 'row': 0}, page_content='survived: 0\npclass: 3\nsex: male\nage: 22.0\nsibsp: 1\nparch: 0\nfare: 7.25\nembarked: S\nclass: Third\nwho: man\nadult_male: True\ndeck: \nembark_town: Southampton\nalive: no\nalone: False'),
 Document(metadata={'source': './data/titanic.csv', 'row': 1}, page_content='survived: 1\npclass: 1\nsex: female\nage: 38.0\nsibsp: 1\nparch: 0\nfare: 71.2833\nembarked: C\nclass: First\nwho: woman\nadult_male: False\ndeck: C\nembark_town: Cherbourg\nalive: yes\nalone: False'),
 Document(metadata={'source': './data/titanic.csv', 'row': 2}, page_content='survived: 1\npclass: 3\nsex: female\nage: 26.0\nsibsp: 0\nparch: 0\nfare: 7.925\nembarked: S\nclass: Third\nwho: woman\nadult_male: False\ndeck: \nembark_town: Southampton\nalive: yes\nalone: True'),
 Document(metadata={'source': './data/titanic.csv', 'row': 3}, page_content='survived: 1\npclass: 1\nsex: female\nage: 35.0\nsibsp: 1\nparch: 0\nfare: 53.1\nembarked: S\ncla

In [102]:
from datasets import load_dataset

# Titanic 데이터셋 로드
df = load_dataset('titanic')

# DataFrame의 인덱스를 리셋하고 새로운 인덱스 열 추가
df = df.reset_index()

# 인덱스 열의 데이터 타입을 문자열로 변환
df['index'] = df['index'].astype(str)

# DataFrameLoader를 사용하여 DataFrame을 문서 객체로 변환
loader = DataFrameLoader(df, page_content_column='index')  # 'index' 열을 페이지 내용으로 사용

# 문서 객체 로드
documents = loader.load()

# 로드된 문서 출력
# documents


[Document(metadata={'survived': 0, 'pclass': 3, 'sex': 'male', 'age': 22.0, 'sibsp': 1, 'parch': 0, 'fare': 7.25, 'embarked': 'S', 'class': 'Third', 'who': 'man', 'adult_male': True, 'deck': nan, 'embark_town': 'Southampton', 'alive': 'no', 'alone': False}, page_content='0'),
 Document(metadata={'survived': 1, 'pclass': 1, 'sex': 'female', 'age': 38.0, 'sibsp': 1, 'parch': 0, 'fare': 71.2833, 'embarked': 'C', 'class': 'First', 'who': 'woman', 'adult_male': False, 'deck': 'C', 'embark_town': 'Cherbourg', 'alive': 'yes', 'alone': False}, page_content='1'),
 Document(metadata={'survived': 1, 'pclass': 3, 'sex': 'female', 'age': 26.0, 'sibsp': 0, 'parch': 0, 'fare': 7.925, 'embarked': 'S', 'class': 'Third', 'who': 'woman', 'adult_male': False, 'deck': nan, 'embark_town': 'Southampton', 'alive': 'yes', 'alone': True}, page_content='2'),
 Document(metadata={'survived': 1, 'pclass': 1, 'sex': 'female', 'age': 35.0, 'sibsp': 1, 'parch': 0, 'fare': 53.1, 'embarked': 'S', 'class': 'First', 'who'

#### Excel

excel 파일을 읽기 위한 방법. <br>
&nbsp;&nbsp;&nbsp;&nbsp; .xls, .xlsx에서 모두 작동

<br>

> ```python
> loader = UnstructuredExcelLoader(<path>, mode='elements')
> loader.load() # 로드
>
> # dataframe loader
> df = pd.read_excel(<path>)
> loader = DataFrameLoader(df, page_content_column="Name")
> loader.load() # 로드
> ```

In [105]:
# Titanic 데이터프레임을 Excel 파일로 저장
df.to_excel('./data/titanic.xlsx', index=False)  # './data/titanic.xlsx' 경로에 저장, 인덱스는 저장하지 않음

In [109]:
# UnstructuredExcelLoader 클래스를 사용하여 Excel 파일 로드
loader = UnstructuredExcelLoader('./data/titanic.xlsx')  # Excel 파일 경로 지정

# Excel 파일에서 문서 내용 로드
documents_single = loader.load()  # 로드된 문서가 documents_single에 저장

# 로드된 문서 출력
documents_single

[Document(metadata={'source': './data/titanic.xlsx'}, page_content='index survived pclass sex age sibsp parch fare embarked class who adult_male deck embark_town alive alone 0.0 0 0 3 male 22 1 0 7.25 S Third man True Southampton no False 1.0 1 1 1 female 38 1 0 71.2833 C First woman False C Cherbourg yes False 2.0 2 1 3 female 26 0 0 7.925 S Third woman False Southampton yes True 3.0 3 1 1 female 35 1 0 53.1 S First woman False C Southampton yes False 4.0 4 0 3 male 35 0 0 8.05 S Third man True Southampton no True 5.0 5 0 3 male 0 0 8.4583 Q Third man True Queenstown no True 6.0 6 0 1 male 54 0 0 51.8625 S First man True E Southampton no True 7.0 7 0 3 male 2 3 1 21.075 S Third child False Southampton no False 8.0 8 1 3 female 27 0 2 11.1333 S Third woman False Southampton yes False 9.0 9 1 2 female 14 1 0 30.0708 C Second child False Cherbourg yes False 10.0 10 1 3 female 4 1 1 16.7 S Third child False G Southampton yes False 11.0 11 1 1 female 58 0 0 26.55 S First woman False C Sout

In [119]:
print(documents_single[0].page_content)

index survived pclass sex age sibsp parch fare embarked class who adult_male deck embark_town alive alone 0.0 0 0 3 male 22 1 0 7.25 S Third man True Southampton no False 1.0 1 1 1 female 38 1 0 71.2833 C First woman False C Cherbourg yes False 2.0 2 1 3 female 26 0 0 7.925 S Third woman False Southampton yes True 3.0 3 1 1 female 35 1 0 53.1 S First woman False C Southampton yes False 4.0 4 0 3 male 35 0 0 8.05 S Third man True Southampton no True 5.0 5 0 3 male 0 0 8.4583 Q Third man True Queenstown no True 6.0 6 0 1 male 54 0 0 51.8625 S First man True E Southampton no True 7.0 7 0 3 male 2 3 1 21.075 S Third child False Southampton no False 8.0 8 1 3 female 27 0 2 11.1333 S Third woman False Southampton yes False 9.0 9 1 2 female 14 1 0 30.0708 C Second child False Cherbourg yes False 10.0 10 1 3 female 4 1 1 16.7 S Third child False G Southampton yes False 11.0 11 1 1 female 58 0 0 26.55 S First woman False C Southampton yes True 12.0 12 0 3 male 20 0 0 8.05 S Third man True South

In [110]:
# UnstructuredExcelLoader 클래스를 사용하여 Excel 파일 로드
loader = UnstructuredExcelLoader('./data/titanic.xlsx', mode='elements')  # Excel 파일 경로와 모드 지정

# Excel 파일에서 문서 내용 로드 (elements 모드)
documents_elements = loader.load()  # 로드된 문서가 documents_elements에 저장

# 로드된 문서 출력
documents_elements

[Document(metadata={'source': './data/titanic.xlsx', 'file_directory': './data', 'filename': 'titanic.xlsx', 'last_modified': '2024-10-24T12:30:02', 'page_name': 'Sheet1', 'page_number': 1, 'text_as_html': '<table><tr><td/><td>index</td><td>survived</td><td>pclass</td><td>sex</td><td>age</td><td>sibsp</td><td>parch</td><td>fare</td><td>embarked</td><td>class</td><td>who</td><td>adult_male</td><td>deck</td><td>embark_town</td><td>alive</td><td>alone</td></tr><tr><td>0.0</td><td>0</td><td>0</td><td>3</td><td>male</td><td>22</td><td>1</td><td>0</td><td>7.25</td><td>S</td><td>Third</td><td>man</td><td>True</td><td/><td>Southampton</td><td>no</td><td>False</td></tr><tr><td>1.0</td><td>1</td><td>1</td><td>1</td><td>female</td><td>38</td><td>1</td><td>0</td><td>71.2833</td><td>C</td><td>First</td><td>woman</td><td>False</td><td>C</td><td>Cherbourg</td><td>yes</td><td>False</td></tr><tr><td>2.0</td><td>2</td><td>1</td><td>3</td><td>female</td><td>26</td><td>0</td><td>0</td><td>7.925</td><td>S<

In [135]:
# documents_elements에서 첫 번째 문서의 HTML 텍스트를 가져와서 BeautifulSoup으로 파싱
BeautifulSoup(documents_elements[0].metadata.get('text_as_html'), 'lxml')  # lxml 파서 사용

<html><body><table><tr><td></td><td>index</td><td>survived</td><td>pclass</td><td>sex</td><td>age</td><td>sibsp</td><td>parch</td><td>fare</td><td>embarked</td><td>class</td><td>who</td><td>adult_male</td><td>deck</td><td>embark_town</td><td>alive</td><td>alone</td></tr><tr><td>0.0</td><td>0</td><td>0</td><td>3</td><td>male</td><td>22</td><td>1</td><td>0</td><td>7.25</td><td>S</td><td>Third</td><td>man</td><td>True</td><td></td><td>Southampton</td><td>no</td><td>False</td></tr><tr><td>1.0</td><td>1</td><td>1</td><td>1</td><td>female</td><td>38</td><td>1</td><td>0</td><td>71.2833</td><td>C</td><td>First</td><td>woman</td><td>False</td><td>C</td><td>Cherbourg</td><td>yes</td><td>False</td></tr><tr><td>2.0</td><td>2</td><td>1</td><td>3</td><td>female</td><td>26</td><td>0</td><td>0</td><td>7.925</td><td>S</td><td>Third</td><td>woman</td><td>False</td><td></td><td>Southampton</td><td>yes</td><td>True</td></tr><tr><td>3.0</td><td>3</td><td>1</td><td>1</td><td>female</td><td>35</td><td>1</td>

#### Word

word 파일을 읽기 위한 방법. <br>

<br>

> ```python
> loader = Docx2txtLoader(<path>)
> loader.load()
>
> # Unstructured
> loader = UnstructuredWordDocumentLoader(<path>, mode='elements')
> loader.load() # 로드
> ```

#### PowerPoint

ppt 파일을 읽기 위한 방법. <br>

<br>

> ```python
> loader = UnstructuredPowerPointLoader(<path>, mode='elements')
> loader.load() # 로드
> ```

In [136]:
# UnstructuredPowerPointLoader 클래스를 사용하여 PowerPoint 파일 로드
loader = UnstructuredPowerPointLoader('./data/231129_안재한.pptx')  # PowerPoint 파일 경로 지정

# PowerPoint 파일에서 문서 내용 로드
documents = loader.load()  # 로드된 문서가 documents 변수에 저장

# 로드된 문서 출력
documents

In [139]:
documents[0].page_content

'Fast Feedforward Networks\n\n23-11-29\n\nJaehan Ahn\n\nInformation & Intelligence System Lab.\n\nSungkyunkwan University\n\narXiv (https://arxiv.org/abs/2308.14711v2)\n\n\n\nBackgrounds\n\n\n\nProposed Method\n\n\n\nProposed Method\n\nTrain\n\nInference\n\n\n\nProposed Method\n\n\n\nProblems\n\nOverfragmentation\n\nLocalized overfitting\n\nWhen node is too hardened before leaf is trained enough\n\n-> Randomized child transposition\n\nshrinking batch problem\n\nWhen the batch size is too small, each leaf receives gradient from small fraction of data\n\nGradient descent becomes unstable and inaccurate\n\n-> Large batch size, gradient accumulation, smaller learning rate\n\n\n\nComplexity\n\n\n\nExperiments (accuracy and speedup)\n\nTotal width\n\ntrain\n\ntest\n\ntrain\n\ntest\n\ntrain\n\ntest\n\ntrain\n\ntest\n\n\n\nExperiments (same inference budget)\n\ntrain\n\ntest\n\nInference size(leaf+node)->\n\nVanilla FF\n\n\n\nExperiments (epochs to achieve accuracy)\n\nRequired Epoch\n\n\n\nEx

In [140]:
# UnstructuredPowerPointLoader 클래스를 사용하여 PowerPoint 파일 로드
loader = UnstructuredPowerPointLoader('./data/231129_안재한.pptx', mode='elements')  # PowerPoint 파일 경로와 모드 지정

# PowerPoint 파일에서 문서 내용을 로드 (elements 모드)
documents = loader.load()  # 로드된 문서가 documents 변수에 저장

# 로드된 문서 출력
documents

In [157]:
# 1페이지에 있는 내용 결합
# -> 페이지 단위로 결합
'\n'.join([document.page_content for document in documents if document.metadata.get('page_number')==1])

'Fast Feedforward Networks\n23-11-29\nJaehan Ahn\nInformation & Intelligence System Lab.\nSungkyunkwan University\narXiv (https://arxiv.org/abs/2308.14711v2)\n'

In [162]:
# 페이지별로 문서 내용을 결합할 딕셔너리 초기화
joined_documents = {}

# 문서의 총 페이지 수를 기준으로 반복
for page in range(1, documents[-1].metadata.get('page_number') + 1):
    # 각 페이지에 해당하는 문서 내용을 결합
    joined_documents.update({
        page: '\n'.join(
            [document.page_content  # 문서의 페이지 내용을 가져옴
            for document in documents
            if document.metadata.get('page_number') == page  # 현재 페이지와 일치하는 문서 필터링
            ]
        )
    })

# 결합된 문서 출력
print(joined_documents)

#### WebBaseLoader

웹 문서 로드 <br>

bs4.SoupStrainer 를 사용하여 파싱할 요소를 지정. <br>
bs_kwargs를 사용하여 bs4.SoupStrainer의 인수 지정. <br>

<br>

> ```python
> loader = WebBaseLoader(
>     web_paths=[
>         <url1>,
>         <url2>,
>     ],
>     bs_kwargs=dict(
>         parse_only=bs4.SoupStrainer(
>             'div',
>             attrs={'class': ['newsct_article _article_body', 'media_end_head_title']},
>         )
>     ),
>     header_template={
>         'User-Agent': 'Mozilla/5.0',
>     },
>     proxies={   # ip 차단 우회를 위해 사용
>         <proxy1>,
>         <proxy2>,
>     }
> )
> 
> loader.requests_kwargs = {"verify": False}  # ssl 인증 우회
> docs = loader.load()
> 
> # 비동기
> nest_asyncio.apply()    # notebook에서만 실행
> loader.requests_per_second = 1
> docs = loader.load()
> ```


In [177]:
from langchain.document_loaders import WebBaseLoader
import bs4

# 웹 페이지 로더 초기화
loader = WebBaseLoader(
    web_paths=[
        'https://n.news.naver.com/article/055/0001200190?cds=news_media_pc&type=breakingnews',
        'https://n.news.naver.com/mnews/article/421/0007865604',
        'https://n.news.naver.com/mnews/article/014/0005257877',
        'https://n.news.naver.com/mnews/article/003/0012861683',
        'https://n.news.naver.com/mnews/article/001/0015004146',
        'https://n.news.naver.com/mnews/article/018/0005867017',
        'https://n.news.naver.com/mnews/article/014/0005257868',
        'https://n.news.naver.com/mnews/article/015/0005048418',
        'https://n.news.naver.com/mnews/article/009/0005384689',
        'https://n.news.naver.com/mnews/article/031/0000879056',
        'https://n.news.naver.com/mnews/article/015/0005048416',
    ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            'div',  # 'div' 태그를 파싱
            attrs={'class': ['media_end_head_title', 'newsct_article _article_body']}  # 특정 클래스에 해당하는 요소만 선택
        )
    ),
    header_template={
        'User-Agent': 'Mozilla/5.0'  # HTTP 요청 시 사용자 에이전트 헤더 설정
    }
)

# 웹 페이지에서 문서 내용 로드
documents = loader.load()

# 로드된 문서 출력
documents

[Document(metadata={'source': 'https://n.news.naver.com/article/055/0001200190?cds=news_media_pc&type=breakingnews'}, page_content='\n[속보] LG전자 3분기 영업이익 20.9% 감소…매출은 최대\n\n\n\n\n\n\n▲ 서울 영등포구 여의도 LG전자 사옥\n\nLG전자가 올해 3분기 물류비 인상 등의 영향으로 영업이익이 20%가량 감소했습니다.\n\n다만 사업 포트폴리오 고도화로 매출 성장세를 이어가며 매출은 3분기 기준 최대치를 기록했습니다.\n\nLG전자는 연결 기준 올해 3분기 영업이익이 7,519억 원으로, 지난해 같은 기간보다 20.9% 감소한 것으로 잠정 집계됐다고 공시했습니다.\n\n이는 시장 전망치 9,686억 원을 22.4% 밑도는 수준입니다.\n\n매출은 22조 1,764억 원으로 지난해 동기 대비 10.7% 증가했습니다.\n\n순이익은 902억 원으로 81.4% 줄었습니다.\n\n역대 3분기 실적과 비교하면 매출은 최대치, 영업이익은 4번째로 높은 수치입니다.\n\n기업간거래(B2B) 사업 가속화 등 포트폴리오 고도화를 통해 4개 분기 연속 전년 대비 매출이 성장했습니다.\n\n다만 중동 지역의 지정학적 분쟁, 미국의 대중국 관세 인상 등의 여파로 글로벌 해상 운임이 상승하면서 수익성에 영향을 끼쳤습니다.\n\n사업부별로 보면 생활가전을 맡은 H&A사업본부의 경우 3분기 매출액은 지난해 대비 11.7% 증가한 8조 3,376억 원, 영업이익은 5.5% 증가한 5,272억 원을 기록했습니다.\n\n글로벌 가전 수요 회복 지연에도 B2B 냉난방공조 사업과 가전구독 사업의 빠른 성장이 매출 확대를 이끌었습니다.\n\n연결 영업이익의 70%가 생활가전에서 나왔습니다.\n\nTV 사업을 담당하는 HE사업본부는 3분기 매출액 3조 7,473억 원, 영업이익 494억 원을 기록했습니다.\n\n매출은 올레드 TV 주요 시장인 유럽 지역의 출하량 증

In [178]:
import nest_asyncio

# 기존 이벤트 루프에 중첩을 허용하도록 설정
nest_asyncio.apply()

In [241]:
# 웹 페이지 로더 초기화
loader = WebBaseLoader(
    web_paths=[
        'https://n.news.naver.com/article/055/0001200190?cds=news_media_pc&type=breakingnews',
        'https://n.news.naver.com/mnews/article/421/0007865604',
        'https://n.news.naver.com/mnews/article/014/0005257877',
        'https://n.news.naver.com/mnews/article/003/0012861683',
        'https://n.news.naver.com/mnews/article/001/0015004146',
        'https://n.news.naver.com/mnews/article/018/0005867017',
        'https://n.news.naver.com/mnews/article/014/0005257868',
        'https://n.news.naver.com/mnews/article/015/0005048418',
        'https://n.news.naver.com/mnews/article/009/0005384689',
        'https://n.news.naver.com/mnews/article/031/0000879056',
        'https://n.news.naver.com/mnews/article/015/0005048416',
    ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            'div',  # 'div' 태그만 파싱
            attrs={
                'class': [
                    'media_end_head_title',  # 제목
                    'newsct_article _article_body',  # 기사 본문
                    'media_end_head_info nv_notrans ',  # 헤드 정보
                ]},
        )
    ),
    header_template={
        'User-Agent': 'Mozilla/5.0'  # HTTP 요청 시 사용자 에이전트 설정
    },
)

# 초당 요청 수 설정
loader.request_per_second = 11

# SSL 인증서 검증 비활성화
loader.requets_kwargs = {'verify': False}

# 웹 페이지에서 문서 내용 비동기로 로드
documents = loader.aload()

# 로드된 문서 출력
documents

Fetching pages: 100%|##########| 11/11 [00:00<00:00, 26.30it/s]


[Document(metadata={'source': 'https://n.news.naver.com/article/055/0001200190?cds=news_media_pc&type=breakingnews'}, page_content='\nLG전자 3분기 영업이익 20.9% 감소…매출은 최대\n\n\n\n입력2024.10.24. 오후 2:04\n\n\n수정2024.10.24. 오후 2:36\n\n기사원문\n \n\n\n\n\n김지성 기자\n\n\n\n\n\n\n\n\n\n\n김지성 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\n\n▲ 서울 영등포구 여의도 LG전자 사옥\n\nLG전자가 올해 3분기 물류비 인상 등의 영향으로 영업이익이 20%가량 감소했습니다.\n\n다만 사업 포트폴리오 고도화로 매출 성장세를 이어가며 매출은 3분기 기준 최대치

#### TextLoader

.txt 확장자를 가지는 파일 로드

<br>

> ```python
> loader = loader = TextLoader(<path>)
> 
> documents = loader.load()   # 전체 문서 로드
> documents = loader.get_summaries_as_docs()  # 요약 문서 로드
> ```

<br>

디렉토리 내 대량 파일 로드 <br>

```python
loader = DirectoryLoader(
    <path>,
    glob='**/*.txt',
    loader_cls=TextLoader,
    silent_errors=True, # 로드 불가 파일 skip
    loader_kwargs={'autodetect_encoding': True},    # encoding 자동 감지
)
docs = loader.load()
```

In [193]:
# TextLoader 클래스를 사용하여 './data/keywords.txt' 파일을 로드합니다.
loader = TextLoader('./data/keywords.txt')

# 로드한 파일의 내용을 documents 변수에 저장합니다.
documents = loader.load()


In [197]:
# DirectoryLoader 클래스를 사용하여 './data' 디렉토리에서 .txt 파일들을 로드합니다.
loader = DirectoryLoader(
    './data',                      # 파일을 로드할 디렉토리 경로
    glob='*.txt',                 # 로드할 파일의 패턴 (여기서는 모든 .txt 파일)
    loader_cls=TextLoader,        # 각 파일을 로드할 때 사용할 클래스
    silent_errors=True,           # 오류 발생 시 조용히 처리하도록 설정
    loader_kwargs={'autodetect_encoding': True},  # 로더에 전달할 추가 인수 (인코딩 자동 감지)
)

# 설정된 로더를 사용하여 파일들을 로드하고, 결과를 documents 변수에 저장합니다.
documents = loader.load()

# 로드된 documents 변수를 출력하여 내용을 확인합니다.
documents

[Document(metadata={'source': 'data\\keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝\n\nAttention 메커니즘\n\n정의: Attention 메커니즘은 딥러닝에서 중요한 정보에 더 많은 \'주의\'를 기울이도록 하는 기법입니다.\n예시: 번역 모델에서 Attention 메커니즘은 입력 문장의 중요한 부분에 더 집중하여 정확한 번역을 생성합니다.\n연관키워드: 딥러닝, 자연어 처리, 시퀀스 모델링\n\n판다스 (Pandas)\n\n정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다.\n예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.\n연관키워드: 데이터 분석, 파이썬, 데이터 처리\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nDataFrame\n\n정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.\n예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이터 조작과 분석을 용이하게 합니다.\n연관키워드: 데이터 분석, 판다스, 데이터 처리\n\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의

#### JSON

json 파일 로드

<br>

> ```python
> loader = JSONLoader(
>     file_path=<path>,
>     jq_schema='.[].phoneNumbers',
>     text_content=False,
> )
> 
> documents = loader.load()   # 전체 문서 로드
> documents = loader.get_summaries_as_docs()  # 요약 문서 로드
> ```

<br>

ex) jq_schema
> ```
> [
>     {
>         "name": "Alice",
>         "phoneNumbers": ["123-456-7890", "987-654-3210"]
>     },
>     {
>         "name": "Bob",
>         "phoneNumbers": ["555-555-5555"]
>     }
> ]
> ```

-> ["123-456-7890", "987-654-3210", "555-555-5555"]와 같은 배열을 반환

In [225]:
# JSONLoader 클래스를 사용하여 './data/phone_numbers.json' 파일에서 데이터를 로드합니다.
loader = JSONLoader(
    file_path='./data/phone_numbers.json',  # 로드할 JSON 파일의 경로
    jq_schema='.[].name',                    # JQ 쿼리를 사용하여 각 항목의 'name' 필드를 선택
    text_content=False                       # 텍스트 내용은 로드하지 않도록 설정
)

# 설정된 로더를 사용하여 JSON 데이터를 로드합니다.
loader.load()

[Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\phone_numbers.json', 'seq_num': 1}, page_content='Alice'),
 Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\phone_numbers.json', 'seq_num': 2}, page_content='Bob')]

In [229]:
# JSONLoader 클래스를 사용하여 './data/test.json' 파일에서 데이터를 로드합니다.
loader = JSONLoader(
    file_path='./data/test.json',            # 로드할 JSON 파일의 경로
    jq_schema='.[].role, .[].content',       # JQ 쿼리를 사용하여 각 항목의 'role'과 'content' 필드를 선택
    text_content=False                        # 텍스트 내용은 로드하지 않도록 설정
)

# 설정된 로더를 사용하여 JSON 데이터를 로드합니다.
loader.load()

[Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\test.json', 'seq_num': 1}, page_content='system'),
 Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\test.json', 'seq_num': 2}, page_content='user'),
 Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\test.json', 'seq_num': 3}, page_content='assistant'),
 Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\test.json', 'seq_num': 4}, page_content='기업공시 요약을 위한 봇이다.'),
 Document(metadata={'source': 'C:\\Users\\USER\\Documents\\git\\2024-08-05_Encore\\day52\\data\\test.json', 'seq_num': 5}, page_content="1. 회사의 개요 가. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 사업보고서, 분기ㆍ반기보고서를 제출하는 경우에 한함)※ 연결대상 종속회사 현황(요약) 1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 사업보고서, 분기ㆍ반기보고서를 제출하는 경우에 한함)연결대상 종속회사 현황(요약) (단위 : 사) 구분 연결대상회사수 주요종속회사수 기초 증가 감소 기말 상장 - - - - - 비상장 5 - - 5 - 합계 5 - - 5 - ※상세 현황

#### Arxiv

arxiv에서 데이터 접근
- arxiv: 수학, 컴퓨터 과학 등 학술 논문을 위한 오픈 엑세스 아카이브

<br>

> ```python
> loader = ArxivLoader(
>     query='Deep Learning',
>     load_max_docs=10,  # 최대 문서 수
>     load_all_available_meta=False,  # 메타데이터 전체 로드 여부
> )
> 
> documents = loader.load()   # 전체 문서 로드
> documents = loader.get_summaries_as_docs()  # 요약 문서 로드
> 

In [205]:
# ArxivLoader 클래스를 사용하여 'large language model'에 대한 논문을 로드합니다.
loader = ArxivLoader(
    query='large language model',  # Arxiv에서 검색할 쿼리
    load_max_docs=10,              # 최대 10개의 문서를 로드하도록 설정, 현재 동작하지 않음
)

# 설정된 로더를 사용하여 논문 데이터를 로드하고, 결과를 documents 변수에 저장합니다.
documents = loader.load()


In [206]:
documents

[Document(metadata={'Published': '2023-06-12', 'Title': 'Lost in Translation: Large Language Models in Non-English Content Analysis', 'Authors': 'Gabriel Nicholas, Aliya Bhatia', 'Summary': "In recent years, large language models (e.g., Open AI's GPT-4, Meta's LLaMa,\nGoogle's PaLM) have become the dominant approach for building AI systems to\nanalyze and generate language online. However, the automated systems that\nincreasingly mediate our interactions online -- such as chatbots, content\nmoderation systems, and search engines -- are primarily designed for and work\nfar more effectively in English than in the world's other 7,000 languages.\nRecently, researchers and technology companies have attempted to extend the\ncapabilities of large language models into languages other than English by\nbuilding what are called multilingual language models.\n  In this paper, we explain how these multilingual language models work and\nexplore their capabilities and limits. Part I provides a simple

## 텍스트 분할

site: https://chunkviz.up.railway.app/

### CharacterTextSplitter

기본적으로 "\n\n" 을 기준으로 문자 단위로 텍스트 분할. <br>
청크의 크기를 문자 수로 측정. <br>

- 텍스트 분할 방식: 단일 문자
- 청크 크기 측정 방식: 문자 수

<br>

> ```python
> with open(<path>) as file:
>     file = file.read()
> 
> text_splitter = CharacterTextSplitter(
>     separator='\n\n',              # 텍스트를 분할할 때 사용할 구분자
>     chunk_size=250,               # 분할된 텍스트 chunk의 최대 크기
>     chunk_overlap=50,             # 분할된 텍스트 chunk 간 중복되는 문자 수
>     length_function=len,          # 텍스트의 길이를 계산하는 함수
>     is_separator_regex=False,      # 구분자가 정규식인지 여부
> )
> 
> metadatas = [
>     {'document1': 1},
> ]
> 
> documents = text_splitter.create_documents(
>     [file],
>     metadatas=metadatas,
> )
> ```

In [246]:
# documents 리스트에서 파일들을 files 변수에 저장합니다.
files = documents

# 첫 번째 문서의 페이지 내용을 file1 변수에 저장합니다.
file1 = files[0].page_content

# 두 번째 문서의 페이지 내용을 file2 변수에 저장합니다.
file2 = files[1].page_content

In [251]:
# CharacterTextSplitter 클래스를 사용하여 텍스트를 분할하는 설정을 정의합니다.
text_splitter = CharacterTextSplitter(
    separator='\n',            # 각 청크를 분할할 구분자 (여기서는 줄바꿈)
    chunk_size=300,           # 각 청크의 최대 크기 (300 문자)
    chunk_overlap=100,        # 청크 간의 겹치는 문자 수 (100 문자)
    length_function=len,      # 텍스트 길이를 측정할 함수 (여기서는 기본 len 함수 사용)
)

# 문서 메타데이터를 정의합니다. (여기서는 문서 ID를 포함)
metadatas = [
    {'document': 1},          # 첫 번째 문서 메타데이터
    {'document': 2},          # 두 번째 문서 메타데이터
]

# file1과 file2의 내용을 청크로 분할하고, 메타데이터를 추가하여 새로운 문서 리스트를 생성합니다.
documents = text_splitter.create_documents(
    [file1, file2],           # 분할할 텍스트 리스트
    metadatas=metadatas,      # 각 문서에 대한 메타데이터
)

# 생성된 documents 변수를 출력하여 내용을 확인합니다.
documents

Created a chunk of size 1002, which is longer than the specified 300


[Document(metadata={'document': 1}, page_content='LG전자 3분기 영업이익 20.9% 감소…매출은 최대\n입력2024.10.24. 오후 2:04\n수정2024.10.24. 오후 2:36\n기사원문\n \n김지성 기자\n김지성 기자\n구독\n구독중\n구독자\n0\n응원수\n0\n더보기\n추천\n쏠쏠정보\n0\n흥미진진\n0\n공감백배\n0\n분석탁월\n0\n후속강추\n0\n \n댓글\n본문 요약봇\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n텍스트 음성 변환 서비스 사용하기\n성별\n남성\n여성'),
 Document(metadata={'document': 1}, page_content='닫기\n텍스트 음성 변환 서비스 사용하기\n성별\n남성\n여성\n말하기 속도\n느림\n보통\n빠름\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n닫기\n \n글자 크기 변경하기\n가1단계\n작게\n가2단계\n보통\n가3단계\n크게\n가4단계\n아주크게\n가5단계\n최대크게\nSNS 보내기\n인쇄하기\n▲ 서울 영등포구 여의도 LG전자 사옥\nLG전자가 올해 3분기 물류비 인상 등의 영향으로 영업이익이 20%가량 감소했습니다.\n다만 사업 포트폴리오 고도화로 매출 성장세를 이어가며 매출은 3분기 기준 최대치를 기록했습니다.'),
 Document(metadata={'document': 1}, page_content='다만 사업 포트폴리오 고도화로 매출 성장세를 이어가며 매출은 3분기 기준 최대치를 기록했습니다.\nLG전자는 연결 기준 올해 3분기 영업이익이 7,519억 원으로, 지난해 같은 기간보다 20.9% 감소한 것으로 잠정 집계됐다고 공시했습니다.\n이는 시장 전망치 9,686억 원을 22.4% 밑도는 수준입니다.\n매출은 22조 1,7

### RecursiveCharacterTextSplitter

일반적인 텍스트에 권장되는 방식. <br>
문자 목록을 매개변수로 받아 동작. <br>
chunk가 충분히 작아질 때까지 주어진 문자 목록의 순서대로 텍스트 분할 시도.
&nbsp;&nbsp;&nbsp;&nbsp; 기본 문자 목록: ["\n\n", "\n", " ", ""] <br>
단락 -> 문장 -> 단어 순서로 재귀적으로 분할. <br>
-> 이는 순서대로 단위가 의미적으로 가장 강하게 연관된 텍스트 조각으로 간주되어, 가능한 한 함께 유지하려는 효과가 있음. <br>
문자 수에 의해 청크 크기 측정. <br>

<br>

> ```python
> with open(<path>) as file:
>     file = file.read()
> 
> text_splitter = RecursiveCharacterTextSplitter(
>     chunk_size=250,               # 분할된 텍스트 chunk의 최대 크기
>     chunk_overlap=50,             # 분할된 텍스트 chunk 간 중복되는 문자 수
>     length_function=len,          # 텍스트의 길이를 계산하는 함수
>     is_separator_regex=False,     # 구분자가 정규식인지 여부
> )
> 
> metadatas = [
>     {'document1': 1},
> ]
> 
> documents = text_splitter.create_documents([file])
> ```

In [252]:
# RecursiveCharacterTextSplitter 클래스를 사용하여 텍스트를 분할하는 설정을 정의합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,            # 각 청크의 최대 크기 (300 문자)
    chunk_overlap=100,         # 청크 간의 겹치는 문자 수 (100 문자)
    length_function=len,       # 텍스트 길이를 측정할 함수 (여기서는 기본 len 함수 사용)
    is_separator_regex=False    # 구분자를 정규 표현식으로 사용할지 여부 (여기서는 사용하지 않음)
)

# file1의 내용을 청크로 분할하여 새로운 문서 리스트를 생성합니다.
documents = text_splitter.create_documents([file1])

# 생성된 documents 변수를 출력하여 내용을 확인합니다.
documents

[Document(metadata={}, page_content='LG전자 3분기 영업이익 20.9% 감소…매출은 최대\n\n\n\n입력2024.10.24. 오후 2:04\n\n\n수정2024.10.24. 오후 2:36\n\n기사원문\n \n\n\n\n\n김지성 기자\n\n\n\n\n\n\n\n\n\n\n김지성 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇'),
 Document(metadata={}, page_content='추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름'),
 Document(metadata={}, page_content='텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n

#### Code Splitter

프로그래밍 언어로 작성된 코드를 분할

<br>

> ```python
> python_code = ""
> 
> python_splitter = RecursiveCharacterTextSplitter.from_language(
>     language=Language.PYTHON,
>     chunk_size=100,
>     chunk_overlap=0,
> )
> ```

In [260]:
# 파이썬 코드를 정의하는 문자열입니다.
python_code = '''
def show_stream(response):
    for token in response:
        print(token.content, end='', flush=True)

show_stream(response)
'''

# RecursiveCharacterTextSplitter를 사용하여 파이썬 코드의 청크를 분할하는 설정을 정의합니다.
code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,  # 코드 언어 설정 (여기서는 파이썬)
    chunk_size=120,            # 각 청크의 최대 크기 (120 문자)
    chunk_overlap=0,           # 청크 간의 겹치는 문자 수 (0으로 설정하여 겹치지 않음)
    length_function=len,       # 텍스트 길이를 측정할 함수 (여기서는 기본 len 함수 사용)
)

# python_code를 청크로 분할하여 새로운 문서 리스트를 생성합니다.
documents = code_splitter.create_documents([python_code])

[Document(metadata={}, page_content="def show_stream(response):\n    for token in response:\n        print(token.content, end='', flush=True)"),
 Document(metadata={}, page_content='show_stream(response)')]

### TokenTextSplitter

토큰 수를 기반으로 chunk 생성. <br>

#### tiktoken

tiktoken 은 OpenAI에서 만든 빠른 BPE Tokenizer

<br>

> ```python
> text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
>     chunk_size=300,               # 분할된 텍스트 chunk의 최대 크기
>     chunk_overlap=50,             # 분할된 텍스트 chunk 간 중복되는 문자 수
> )
> documents= text_splitter.split_text(file)
> ```

In [271]:
# CharacterTextSplitter를 사용하여 TikToken 인코더로부터 청크 분할기를 생성합니다.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,            # 각 청크의 최대 크기 (300 문자)
    chunk_overlap=100,         # 청크 간의 겹치는 문자 수 (100 문자)
)

In [272]:
# 설정된 text_splitter를 사용하여 file1의 내용을 청크로 분할하고,
# 새로운 문서 리스트를 생성합니다.
documents = text_splitter.create_documents([file1])

[Document(metadata={}, page_content='LG전자 3분기 영업이익 20.9% 감소…매출은 최대\n\n입력2024.10.24. 오후 2:04\n\n\n수정2024.10.24. 오후 2:36\n\n기사원문\n \n\n\n김지성 기자\n\n\n김지성 기자\n\n구독\n구독중\n\n\n구독자\n0\n\n\n응원수\n0\n\n더보기\n\n\n추천\n\n\n쏠쏠정보\n0\n\n\n흥미진진\n0\n\n\n공감백배\n0\n\n\n분석탁월\n0\n\n\n후속강추\n0\n\n\n \n\n댓글'),
 Document(metadata={}, page_content='추천\n\n\n쏠쏠정보\n0\n\n\n흥미진진\n0\n\n\n공감백배\n0\n\n\n분석탁월\n0\n\n\n후속강추\n0\n\n\n \n\n댓글\n\n본문 요약봇'),
 Document(metadata={}, page_content='공감백배\n0\n\n\n분석탁월\n0\n\n\n후속강추\n0\n\n\n \n\n댓글\n\n본문 요약봇\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기'),
 Document(metadata={}, page_content='텍스트 음성 변환 서비스 사용하기\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n가1단계\n작게\n\n\n가2단계\n보통'),
 Document(metadata={}, page_content='닫기\n\n\n \n\n글자 크기 변경하기\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\nSNS 보내기\n\n인쇄하기\n\n▲ 서울 영등포구 여의도 

#### Hugging Face tokenizer

huggingface library 내의 tokenizer를 통한 text 분할

<br>

> ```python
> text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
>     tokenizer,
>     chunk_size=300,
>     chunk_overlap=50,
> )
> texts = text_splitter.split_text(file)
> ```

In [274]:
# Hugging Face의 사전 훈련된 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained('skt/kobert-base-v1')

# CharacterTextSplitter를 사용하여 Hugging Face 토크나이저로부터 청크 분할기를 생성합니다.
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,        # 사용할 토크나이저
    chunk_size=300,             # 각 청크의 최대 크기 (300 토큰)
    chunk_overlap=100,          # 청크 간의 겹치는 토큰 수 (100 토큰)
)

# 설정된 text_splitter를 사용하여 주어진 file의 내용을 청크로 분할합니다.
documents = text_splitter.split_text(file)

# 생성된 documents 변수를 출력하여 내용을 확인합니다.
documents

['LG전자 3분기 영업이익 20.9% 감소…매출은 최대\n\n입력2024.10.24. 오후 2:04\n\n\n수정2024.10.24. 오후 2:36\n\n기사원문\n \n\n\n김지성 기자\n\n\n김지성 기자\n\n구독\n구독중\n\n\n구독자\n0\n\n\n응원수\n0\n\n더보기\n\n\n추천\n\n\n쏠쏠정보\n0\n\n\n흥미진진\n0\n\n\n공감백배\n0\n\n\n분석탁월\n0\n\n\n후속강추\n0\n\n\n \n\n댓글\n\n본문 요약봇',
 '더보기\n\n\n추천\n\n\n쏠쏠정보\n0\n\n\n흥미진진\n0\n\n\n공감백배\n0\n\n\n분석탁월\n0\n\n\n후속강추\n0\n\n\n \n\n댓글\n\n본문 요약봇\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n텍스트 음성 변환 서비스 사용하기\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름',
 '텍스트 음성 변환 서비스 사용하기\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\nSNS 보내기\n\n인쇄하기\n\n▲ 서울 영등포구 여의도 LG전자 사옥',
 '가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\nSNS 보내기\n\n인쇄하기\n\n▲ 서울 영등포구 여의도 LG전자 사옥\n\nLG전자가 올해 3분기 물류비 인상 등의 영향으로 영업이익이 20%가량 감소했습니다.\n\n다만 사업 포트폴리오 고도화로 매출 성장세를 이어가며 매출은 3분기 기준 최대치를 기록했습니다.',
 '다만

In [284]:
# example: keyword

# './data/keywords.txt' 파일을 로드하고, 첫 번째 문서의 페이지 내용을 file 변수에 저장합니다.
file = TextLoader('./data/keywords.txt').load()[0].page_content

# CharacterTextSplitter를 사용하여 TikToken 인코더로부터 청크 분할기를 생성합니다.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,            # 각 청크의 최대 크기 (300 토큰)
    chunk_overlap=100,         # 청크 간의 겹치는 토큰 수 (100 토큰)
)

# 설정된 text_splitter를 사용하여 file의 내용을 청크로 분할합니다.
documents = text_splitter.split_text(file)

# 생성된 documents 변수를 출력하여 내용을 확인합니다.
documents

Created a chunk of size 307, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300


['GPT (Generative Pretrained Transformer)',
 '정의: GPT는 대규모 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝',
 "Attention 메커니즘\n\n정의: Attention 메커니즘은 딥러닝에서 중요한 정보에 더 많은 '주의'를 기울이도록 하는 기법입니다.\n예시: 번역 모델에서 Attention 메커니즘은 입력 문장의 중요한 부분에 더 집중하여 정확한 번역을 생성합니다.\n연관키워드: 딥러닝, 자연어 처리, 시퀀스 모델링",
 '판다스 (Pandas)\n\n정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다.\n예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.\n연관키워드: 데이터 분석, 파이썬, 데이터 처리',
 'Deep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nDataFrame',
 '정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.\n예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이터 조작과 분석을 용이하게 합니다.\n연관키워드: 데이터 분석, 판다스, 데이터 처리',
 'LLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리

### SemanticChunker

텍스트를 유사성에 기반하여 분할. 

<br>

> ```python
> text_splitter = SemanticChunker(OpenAIEmbeddings())
> chunks = text_splitter.split_text(file)
> documents = text_splitter.create_documents([file])
> ```

### MarkdownHeaderTextSplitter

문서를 지정된 헤더 집합에 따라 분할. <br>
각 헤더 그룹 아래의 내용을 별도의 chunk로 관리. <br>
-> 문서의 전반적인 구조를 유지하면서도 내용을 더 세밀하게 다룰 수 있음

<br>

> ```python
> markdown_document = ''
> 
> headers_to_split_on = [  # 문서를 분할할 헤더 레벨과 해당 레벨의 이름을 정의.
>     (
>         "#",
>         "Header 1",  # 헤더 레벨 1은 '#'로 표시되며, 'Header 1'이라는 이름을 가짐
>     ),  
>     (
>         "##",
>         "Header 2",  # 헤더 레벨 2는 '##'로 표시되며, 'Header 2'라는 이름을 가짐
>     ),  
>     (
>         "###",
>         "Header 3",  # 헤더 레벨 3은 '###'로 표시되며, 'Header 3'이라는 이름을 가짐
>     ), 
> ]
> 
> markdown_splitter = MarkdownHeaderTextSplitter(
>     headers_to_split_on=headers_to_split_on,
>     strip_headers = False,    # 헤더를 제거하지 않도록 설정
> )
> documents = markdown_splitter.split_text(markdown_document)
>
> # RecursiveCharacterTextSplitter로 분할된 결과 분할
> text_splitter = RecursiveCharacterTextSplitter(
>     chunk_size=200,
>     chunk_overlap=20,
> )
documents = text_splitter.split_documents(md_header_splits)
> ```

In [285]:
markdown = '''
# 딥러닝(Deep Learning)

딥러닝은 **인공지능(AI)**의 한 분야로, **신경망(Neural Networks)**을 기반으로 하는 기계 학습 기술입니다. 이 기술은 복잡한 데이터의 패턴을 학습하고 예측하는 데 매우 효과적입니다.

## 주요 구성 요소

### 신경망(Neural Network)

신경망은 여러 개의 노드(뉴런)로 구성된 구조로, 다음과 같은 층으로 이루어져 있습니다:

- **입력층(Input Layer)**: 모델이 데이터를 받는 첫 번째 층입니다.
- **은닉층(Hidden Layer)**: 입력층과 출력층 사이에 위치하며, 데이터의 특성을 추출합니다.
- **출력층(Output Layer)**: 모델의 최종 결과를 출력합니다.

### 훈련 데이터(Training Data)

훈련 데이터는 신경망이 학습하는 데 사용하는 데이터입니다. 주로 레이블이 있는 데이터가 사용됩니다. 데이터의 품질은 모델의 성능에 큰 영향을 미칩니다.

### 손실 함수(Loss Function)

손실 함수는 모델의 예측값과 실제값 간의 차이를 계산하는 함수입니다. 주로 사용되는 손실 함수는 다음과 같습니다:

- **회귀 문제**: 평균 제곱 오차(Mean Squared Error, MSE)
- **분류 문제**: 교차 엔트로피 손실(Cross-Entropy Loss)

## 딥러닝의 응용 분야

### 이미지 인식

딥러닝은 이미지 인식 분야에서 크게 발전하였으며, 다음과 같은 응용이 있습니다:

- **얼굴 인식**: 보안 시스템에서 사용자 인증에 사용됩니다.
- **자율주행차**: 도로와 물체를 인식하여 안전한 주행을 가능하게 합니다.

### 자연어 처리

자연어 처리 분야에서도 딥러닝은 중요한 역할을 합니다:

- **번역**: 기계 번역 시스템에서 언어를 자동으로 번역합니다.
- **감정 분석**: 소셜 미디어 데이터에서 감정을 분석하여 인사이트를 제공합니다.

### 의료

의료 분야에서도 딥러닝의 활용이 증가하고 있습니다:

- **질병 진단**: 의료 이미지를 분석하여 질병을 조기에 발견합니다.
- **환자 데이터 분석**: 대규모 환자 데이터를 분석하여 치료법을 개선합니다.

## 결론

딥러닝은 다양한 분야에서 혁신을 가져오고 있으며, 앞으로도 계속 발전할 것으로 기대됩니다. 기술의 발전과 함께 윤리적 고려도 중요한 이슈로 떠오르고 있습니다.
'''

In [289]:
# 분할할 헤더를 정의합니다. 각 튜플은 헤더 마크와 해당 헤더의 이름을 포함합니다.
headers_to_split_on = [
    ('#', 'Header 1'),         # 1단계 헤더
    ('##', 'Header 2'),        # 2단계 헤더
    ('###', 'Header 3'),       # 3단계 헤더
]

# MarkdownHeaderTextSplitter를 사용하여 마크다운 텍스트를 헤더를 기준으로 분할하는 설정을 정의합니다.
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,  # 분할할 헤더 목록
    strip_headers=True,                         # 분할 후 헤더를 제거할지 여부
)

# 설정된 markdown_splitter를 사용하여 markdown 텍스트를 분할합니다.
documents = markdown_splitter.split_text(markdown)

# 생성된 documents 변수를 출력하여 내용을 확인합니다.
documents


[Document(metadata={'Header 1': '딥러닝(Deep Learning)'}, page_content='딥러닝은 **인공지능(AI)**의 한 분야로, **신경망(Neural Networks)**을 기반으로 하는 기계 학습 기술입니다. 이 기술은 복잡한 데이터의 패턴을 학습하고 예측하는 데 매우 효과적입니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '신경망(Neural Network)'}, page_content='신경망은 여러 개의 노드(뉴런)로 구성된 구조로, 다음과 같은 층으로 이루어져 있습니다:  \n- **입력층(Input Layer)**: 모델이 데이터를 받는 첫 번째 층입니다.\n- **은닉층(Hidden Layer)**: 입력층과 출력층 사이에 위치하며, 데이터의 특성을 추출합니다.\n- **출력층(Output Layer)**: 모델의 최종 결과를 출력합니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '훈련 데이터(Training Data)'}, page_content='훈련 데이터는 신경망이 학습하는 데 사용하는 데이터입니다. 주로 레이블이 있는 데이터가 사용됩니다. 데이터의 품질은 모델의 성능에 큰 영향을 미칩니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '손실 함수(Loss Function)'}, page_content='손실 함수는 모델의 예측값과 실제값 간의 차이를 계산하는 함수입니다. 주로 사용되는 손실 함수는 다음과 같습니다:  \n- **회귀 문제**: 평균 제곱 오차(Mean Squared Error, MSE)\n- **분류 문제**: 교차

In [288]:
# MarkdownHeaderTextSplitter를 사용하여 마크다운 텍스트를 헤더를 기준으로 분할하는 설정을 정의합니다.
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,  # 분할할 헤더 목록
    strip_headers=False,                        # 분할 후 헤더를 유지할지 여부 (여기서는 유지)
)

# 설정된 markdown_splitter를 사용하여 markdown 텍스트를 분할합니다.
documents = markdown_splitter.split_text(markdown)

# 생성된 documents 변수를 출력하여 내용을 확인합니다.
documents

[Document(metadata={'Header 1': '딥러닝(Deep Learning)'}, page_content='# 딥러닝(Deep Learning)  \n딥러닝은 **인공지능(AI)**의 한 분야로, **신경망(Neural Networks)**을 기반으로 하는 기계 학습 기술입니다. 이 기술은 복잡한 데이터의 패턴을 학습하고 예측하는 데 매우 효과적입니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '신경망(Neural Network)'}, page_content='## 주요 구성 요소  \n### 신경망(Neural Network)  \n신경망은 여러 개의 노드(뉴런)로 구성된 구조로, 다음과 같은 층으로 이루어져 있습니다:  \n- **입력층(Input Layer)**: 모델이 데이터를 받는 첫 번째 층입니다.\n- **은닉층(Hidden Layer)**: 입력층과 출력층 사이에 위치하며, 데이터의 특성을 추출합니다.\n- **출력층(Output Layer)**: 모델의 최종 결과를 출력합니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '훈련 데이터(Training Data)'}, page_content='### 훈련 데이터(Training Data)  \n훈련 데이터는 신경망이 학습하는 데 사용하는 데이터입니다. 주로 레이블이 있는 데이터가 사용됩니다. 데이터의 품질은 모델의 성능에 큰 영향을 미칩니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '손실 함수(Loss Function)'}, page_content='### 손실 함수(Loss Function)  \n손실 함수는 

In [291]:
# RecursiveCharacterTextSplitter를 사용하여 텍스트를 분할하는 설정을 정의합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,            # 각 청크의 최대 크기 (300 문자)
    chunk_overlap=100,         # 청크 간의 겹치는 문자 수 (100 문자)
)

# 설정된 text_splitter를 사용하여 documents 리스트의 내용을 분할합니다.
splits = text_splitter.split_documents(documents)

# 생성된 splits 변수를 출력하여 내용을 확인합니다.
splits

[Document(metadata={'Header 1': '딥러닝(Deep Learning)'}, page_content='딥러닝은 **인공지능(AI)**의 한 분야로, **신경망(Neural Networks)**을 기반으로 하는 기계 학습 기술입니다. 이 기술은 복잡한 데이터의 패턴을 학습하고 예측하는 데 매우 효과적입니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '신경망(Neural Network)'}, page_content='신경망은 여러 개의 노드(뉴런)로 구성된 구조로, 다음과 같은 층으로 이루어져 있습니다:  \n- **입력층(Input Layer)**: 모델이 데이터를 받는 첫 번째 층입니다.\n- **은닉층(Hidden Layer)**: 입력층과 출력층 사이에 위치하며, 데이터의 특성을 추출합니다.\n- **출력층(Output Layer)**: 모델의 최종 결과를 출력합니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '훈련 데이터(Training Data)'}, page_content='훈련 데이터는 신경망이 학습하는 데 사용하는 데이터입니다. 주로 레이블이 있는 데이터가 사용됩니다. 데이터의 품질은 모델의 성능에 큰 영향을 미칩니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '손실 함수(Loss Function)'}, page_content='손실 함수는 모델의 예측값과 실제값 간의 차이를 계산하는 함수입니다. 주로 사용되는 손실 함수는 다음과 같습니다:  \n- **회귀 문제**: 평균 제곱 오차(Mean Squared Error, MSE)\n- **분류 문제**: 교차

### HTMLHeaderTextSplitter

텍스트를 요소 수준에서 분할하고 각 헤더에 대한 메타데이터를 추가하는 구조 인식 chunk 생성. <br>
-> 각 청크와 관련된 메타데이터를 추가. <br>
요소별로 chunk를 반환하거나 동일한 메타데이터를 가진 요소를 결합 가능. <br>
-> 관련 텍스트를 대략적으로, 의미론적으로 그룹화.
-> 문서 구조에 인코딩된 context의 풍부한 정보를 보존.

<br>

> ```python
> html_data = ''
> headers_to_split_on = [
>     ('h1', 'Header 1'),  # 헤더 태그와 해당 헤더의 이름을 지정.
>     ('h2', 'Header 2'),
>     ('h3', 'Header 3'),
> ]
> 
> html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
> documents = html_splitter.split_text(html_data)
>
> ###################
> 
> chunk_size = 300
> chunk_overlap = 30
> text_splitter = RecursiveCharacterTextSplitter(
>     chunk_size=300,
>     chunk_overlap=30,
> )
> documents = text_splitter.split_documents(documents)
> ```

In [295]:
html = '''
<h1>딥러닝(Deep Learning)</h1>

<p>딥러닝은 <strong>인공지능(AI)</strong>의 한 분야로, <strong>신경망(Neural Networks)</strong>을 기반으로 하는 기계 학습 기술입니다. 이 기술은 복잡한 데이터의 패턴을 학습하고 예측하는 데 매우 효과적입니다.</p>

<h2>주요 구성 요소</h2>

<h3>신경망(Neural Network)</h3>

<p>신경망은 여러 개의 노드(뉴런)로 구성된 구조로, 다음과 같은 층으로 이루어져 있습니다:</p>
<ul>
    <li><strong>입력층(Input Layer)</strong>: 모델이 데이터를 받는 첫 번째 층입니다.</li>
    <li><strong>은닉층(Hidden Layer)</strong>: 입력층과 출력층 사이에 위치하며, 데이터의 특성을 추출합니다.</li>
    <li><strong>출력층(Output Layer)</strong>: 모델의 최종 결과를 출력합니다.</li>
</ul>

<h3>훈련 데이터(Training Data)</h3>

<p>훈련 데이터는 신경망이 학습하는 데 사용하는 데이터입니다. 주로 레이블이 있는 데이터가 사용됩니다. 데이터의 품질은 모델의 성능에 큰 영향을 미칩니다.</p>

<h3>손실 함수(Loss Function)</h3>

<p>손실 함수는 모델의 예측값과 실제값 간의 차이를 계산하는 함수입니다. 주로 사용되는 손실 함수는 다음과 같습니다:</p>
<ul>
    <li><strong>회귀 문제</strong>: 평균 제곱 오차(Mean Squared Error, MSE)</li>
    <li><strong>분류 문제</strong>: 교차 엔트로피 손실(Cross-Entropy Loss)</li>
</ul>

<h2>딥러닝의 응용 분야</h2>

<h3>이미지 인식</h3>

<p>딥러닝은 이미지 인식 분야에서 크게 발전하였으며, 다음과 같은 응용이 있습니다:</p>
<ul>
    <li><strong>얼굴 인식</strong>: 보안 시스템에서 사용자 인증에 사용됩니다.</li>
    <li><strong>자율주행차</strong>: 도로와 물체를 인식하여 안전한 주행을 가능하게 합니다.</li>
</ul>

<h3>자연어 처리</h3>

<p>자연어 처리 분야에서도 딥러닝은 중요한 역할을 합니다:</p>
<ul>
    <li><strong>번역</strong>: 기계 번역 시스템에서 언어를 자동으로 번역합니다.</li>
    <li><strong>감정 분석</strong>: 소셜 미디어 데이터에서 감정을 분석하여 인사이트를 제공합니다.</li>
</ul>

<h3>의료</h3>

<p>의료 분야에서도 딥러닝의 활용이 증가하고 있습니다:</p>
<ul>
    <li><strong>질병 진단</strong>: 의료 이미지를 분석하여 질병을 조기에 발견합니다.</li>
    <li><strong>환자 데이터 분석</strong>: 대규모 환자 데이터를 분석하여 치료법을 개선합니다.</li>
</ul>

<h2>결론</h2>

<p>딥러닝은 다양한 분야에서 혁신을 가져오고 있으며, 앞으로도 계속 발전할 것으로 기대됩니다. 기술의 발전과 함께 윤리적 고려도 중요한 이슈로 떠오르고 있습니다.</p>
'''

In [296]:
headers_to_split_on = [
    ('h1', 'Header 1'),
    ('h2', 'Header 2'),
    ('h3', 'Header 3'),
]

html_splitter = HTMLHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
)
documents = html_splitter.split_text(html)
documents

[Document(metadata={'Header 1': '딥러닝(Deep Learning)'}, page_content='딥러닝은 인공지능(AI)의 한 분야로, 신경망(Neural Networks)을 기반으로 하는 기계 학습 기술입니다. 이 기술은 복잡한 데이터의 패턴을 학습하고 예측하는 데 매우 효과적입니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '신경망(Neural Network)'}, page_content='신경망은 여러 개의 노드(뉴런)로 구성된 구조로, 다음과 같은 층으로 이루어져 있습니다:  \n입력층(Input Layer): 모델이 데이터를 받는 첫 번째 층입니다. 은닉층(Hidden Layer): 입력층과 출력층 사이에 위치하며, 데이터의 특성을 추출합니다. 출력층(Output Layer): 모델의 최종 결과를 출력합니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '훈련 데이터(Training Data)'}, page_content='훈련 데이터는 신경망이 학습하는 데 사용하는 데이터입니다. 주로 레이블이 있는 데이터가 사용됩니다. 데이터의 품질은 모델의 성능에 큰 영향을 미칩니다.'),
 Document(metadata={'Header 1': '딥러닝(Deep Learning)', 'Header 2': '주요 구성 요소', 'Header 3': '손실 함수(Loss Function)'}, page_content='손실 함수는 모델의 예측값과 실제값 간의 차이를 계산하는 함수입니다. 주로 사용되는 손실 함수는 다음과 같습니다:  \n회귀 문제: 평균 제곱 오차(Mean Squared Error, MSE) 분류 문제: 교차 엔트로피 손실(Cross-Entropy Loss)'),
 Document

### RecursiveJsonSplitter

깊이 우선 탐색을 통하여 더 작은 json chunk 생성. <br> 
중첩된 json 객체를 가능한 한 유지하려고 시도. <br>
-> but 청크의 크기를 min_chunk_size와 max_chunk_size 사이로 유지하기 위해 필요한 경우 객체 분할. 
값이 중첩된 json이 아니라 매우 큰 문자열인 경우, 해당 문자열은 분할되지 않음. <br
chunk 크기에 제한이 필요한 경우, RecursiveTextSplitter를 사용하여 해당 청크를 처리하는 것을 고려해 볼 수 있음. <br>

분할 기준:
- 텍스트 분할 방식: json 값
- 청크 크기 측정 방식: 문자 수

> ```python
> json_data = ''
> splitter = RecursiveJsonSplitter(max_chunk_size=300)
> json_chunks = splitter.split_json(
>     json_data=json_data,
>     onvert_lists=True,    # 리스트를 index:item 형태의 key:value 쌍으로 변환
>     )
> 
> # json data 기반 문서 생성
> documents = splitter.create_documents(texts=[json_data])
> 
> # json data 기반 문자열 청크 생성
> texts = splitter.split_text(json_data=json_data)
> ```

In [301]:
# JSON 형식의 문자열을 파이썬 딕셔너리로 변환합니다.
json_data = json.loads(''' 
{
    "name": "Alice",               # 이름
    "age": 30,                     # 나이
    "address": {                   # 주소 정보
        "street": "123 Main St",   # 거리
        "city": "Wonderland",      # 도시
        "postalCode": "12345"      # 우편번호
    },
    "hobbies": ["reading", "traveling", "swimming"],  # 취미 목록
    "employment": {                # 고용 정보
        "company": "Tech Co",      # 회사명
        "position": "Engineer"      # 직책
    }
}''')

In [306]:
# RecursiveJsonSplitter를 사용하여 JSON 데이터를 분할하는 설정을 정의합니다.
json_splitter = RecursiveJsonSplitter(max_chunk_size=50)  # 최대 청크 크기를 50으로 설정

# 설정된 json_splitter를 사용하여 json_data를 분할합니다.
# convert_lists=True는 리스트 항목을 개별적으로 분할하도록 설정합니다.
json_chunks = json_splitter.split_json(
    json_data=json_data,         # 분할할 JSON 데이터
    convert_lists=True,          # 리스트를 개별 항목으로 변환
)

# 생성된 json_chunks 변수를 출력하여 내용을 확인합니다.
json_chunks

[{'name': 'Alice', 'age': 30, 'address': {'street': '123 Main St'}},
 {'address': {'city': 'Wonderland', 'postalCode': '12345'}},
 {'hobbies': {'0': 'reading', '1': 'traveling', '2': 'swimming'}},
 {'employment': {'company': 'Tech Co', 'position': 'Engineer'}}]

In [304]:
# RecursiveJsonSplitter를 사용하여 JSON 데이터를 분할하는 설정을 정의합니다.
json_splitter = RecursiveJsonSplitter(max_chunk_size=300)  # 최대 청크 크기를 300으로 설정

# 설정된 json_splitter를 사용하여 json_data를 분할합니다.
json_chunks = json_splitter.split_json(
    json_data=json_data,  # 분할할 JSON 데이터
)

# 생성된 json_chunks 변수를 출력하여 내용을 확인합니다.
json_chunks

[{'name': 'Alice',
  'age': 30,
  'address': {'street': '123 Main St',
   'city': 'Wonderland',
   'postalCode': '12345'},
  'hobbies': ['reading', 'traveling', 'swimming'],
  'employment': {'company': 'Tech Co', 'position': 'Engineer'}}]

### Embedding

한국어 검색을 위한 임베딩 벤치마크: https://github.com/teddylee777/Kor-IR?tab=readme-ov-file

#### OpenAIEmbeddings

OpenAI의 embeding API를 사용한 embedding.

<br>

> ```python
> embedding_model = OpenAIEmbeddings(
>     model="text-embedding-3-small",
>     # dimensions=1024, # 차원 조정
> )
> 
> # text embedding
> embedded_text = embedding_model.embed_query(text)
> 
> # document embedding
> embedded_documents = embeddings.embed_documents([text])
> ```

#### CacheBackedEmbeddings

재계산을 피하기 위해 저장하거나 일시적으로 caching. <br>
embeddings을 key-value 저장소에 caching하는 wrapper. <br>
텍스트는 해시되고 이 해시는 캐시에서 키로 사용. <br>

<br>

CacheBackedEmbeddings:
- underlying_embeddings: embedding을 위해 사용하는 모델.
- document_embedding_cache: 문서 임베딩을 캐싱하기 위한 ByteStore.
- namespace (optional): 다른 캐시와의 충돌을 피하기 위해 사용하는 네임스페이스.

<br>

> ```python
> store = LocalFileStore("./cache/")    # 영구적
> store = InMemoryByteStore()           # 비영구적
> 
> cached_embedder = CacheBackedEmbeddings.from_bytes_store(
>     underlying_embeddings=embedding,
>     document_embedding_cache=store,
>     namespace=embedding.model,
> )
> ```

#### HuggingFace Embeddings

huggingface endpoint를 통한 embedding. <br>

<br>

> ```python
> embedding_model = OllamaEmbeddings(
>     model=<model_name>,
>     task='feature-extraction',
>     huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
>     # model_kwargs: optional ={'device': 'cuda'},  # cuda, cpu
>     # encode_kwargs={'normalize_embeddings': True},
> )
> 
> # 문서 임베딩
> embedded_documents = embedding_model.embed_documents(texts)
> ```

#### OllamaEmbeddings

ollama를 통한 embedding. <br>

<br>

> ```python
> ollama_embeddings = OllamaEmbeddings(
>     model=<model_name>,
> )
> 
> # 문서 임베딩
> embedded_documents = ollama_embeddings.embed_documents(texts)
> ```